In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import config
import requests
import pandas as pd
import csv
import os
from datetime import datetime, date, timedelta

In [3]:
from Base import Base
from NewCurrency import NewCurrency

In [4]:
import Analytics


In [5]:
from __future__ import division

In [105]:
import itertools

## Sections:

1. Get data (right below)
1. [Aggregate, get min, max, std. Filter result by index](#Aggregate---mix,-max,-std---filter-by-index)

In [117]:
base_obj = Base()
# r = base_obj.request_ticker()
# print r

dag_hist = base_obj.get_trade_history('CARD')

In [118]:
dag_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
amount             5000 non-null object
buyerFee           5000 non-null object
date               5000 non-null object
gasFee             5000 non-null object
maker              5000 non-null object
orderHash          5000 non-null object
price              5000 non-null object
sellerFee          5000 non-null object
taker              5000 non-null object
timestamp          5000 non-null int64
total              5000 non-null object
transactionHash    5000 non-null object
type               5000 non-null object
usdValue           5000 non-null object
uuid               5000 non-null object
dtypes: int64(1), object(14)
memory usage: 586.0+ KB


In [8]:
dag_hist

,amount,buyerFee,date,gasFee,maker,orderHash,price,sellerFee,taker,timestamp,total,transactionHash,type,usdValue,uuid
0,2922688.201833510076086956,2922.688201833510076087,2018-08-15 09:01:01,0.00119,0x0ec24ccb11a7df0526dca527c5a347d23abc320b,0x12bba441aa84e2e09320ea8c230600c4cdacf61e1fe9...,0.000000092,0.000537774629137366,0xc883e538d5552324c6f013224f3bb4dd9eb9e70e,1534323661,0.268887314568682926,0x3049caf7b58baf29752ec681e89b4690f924c913bda4...,sell,188.2211201980780482,bbab8330-a069-11e8-8e01-6b262cbc1cee
1,955850.875867913824312456,1911.701751735827648625,2018-08-15 08:58:43,12044.534656771957101933,0xc362530e8d22f4933a96060b234e7ae289af0abd,0xb9e8f7f85b14d91d3773d25d5f98a57f5e409d683aba...,0.000000098799998,0.000094438064624048,0xcf6d785b63502b80abcfd1bcea012d67f01c1142,1534323523,0.09443806462404813,0x62f2314e5c1b0a59c1c0e30bc3a8bdc4e0bac87da066...,buy,66.106645236833691,69334ca0-a069-11e8-8e01-6b262cbc1cee
2,2151424.159096133863948958,2151.424159096133863949,2018-08-15 08:38:17,0.00119,0xc362530e8d22f4933a96060b234e7ae289af0abd,0xea0ce49124803942df670f4472acaf03c367d1eff4fc...,0.0000000921,0.000396292330105508,0x0f008b53c0de9f6e81110d032e7a0c471d353ffa,1534322297,0.198146165052753905,0xbd06f94fa06cf10251675a59210e1873f0f8f62bb6c0...,sell,138.7023155369277335,8ee39cf0-a066-11e8-8e01-6b262cbc1cee
3,2188539.903890581882672404,2188.539903890581882672,2018-08-15 08:28:35,0.00119,0xc362530e8d22f4933a96060b234e7ae289af0abd,0x99cd176fd3e7c5e46a549270b3d2a3eba31194ed2780...,0.0000000921,0.000403129050296645,0xb50a423ce10b86e660f6646d067ed203bf68da13,1534321715,0.201564525148322545,0x5bce0afe58abcaedcf927ee070fe874b1aae8fcb5bc6...,sell,141.0951676038257815,33fdeb20-a065-11e8-8e01-6b262cbc1cee
4,578424.83237893330637007,1156.84966475786661274,2018-08-15 08:13:56,12032.3559150657228545,0xce39c3059fff13edcbe011f5653fd193f5520fc5,0x2af256a0e17e35e6a4eb42bb83e69cd38bb5cf3304dd...,0.0000000989,0.000057206215922277,0x67f622080e3a258cd57462f545860dc1d000150b,1534320836,0.057206215922276504,0xf142807515f896a8486c40d154d37702944788c52636...,buy,40.0443511455935528,27fba5d0-a063-11e8-8e01-6b262cbc1cee
5,31836217.791890589019211324,63672.435583781178038423,2018-08-15 08:10:29,12032.355915065698363411,0x4a0d383ed4e9277f09f020d22aebd2b828e40650,0x25c33d7227412194941877c9c373928c6fd93cb72c70...,0.0000000989,0.003148601939617979,0x092472d7c20c42a9213a4ad4e602c1371eb127a1,1534320629,3.148601939617979254,0x1c7f779c7f88b07cd1aac3f9a36065280c4a305f80e9...,buy,2204.0213577325854778,ac4c97a0-a062-11e8-8e01-6b262cbc1cee
6,9166330.654440119485139684,18332.661308880238970279,2018-08-15 08:10:29,12032.355915065721438447,0xfbaf8025cbabce61e9d29aa54841c3fe84188c9a,0x42a212ce5890961bbc58221f63a504d1fc8f1951988f...,0.0000000989,0.000906550101724128,0x092472d7c20c42a9213a4ad4e602c1371eb127a1,1534320629,0.906550101724127817,0x7ac7e0d7500a1f4964ae8666df0e43d2a50e7fa03127...,buy,634.5850712068894719,ac526400-a062-11e8-8e01-6b262cbc1cee
7,5055611.729019211325018169,10111.223458038422650036,2018-08-15 08:10:29,12032.355915065722953543,0x36789debf58f03ec3d889d01e0b60af01c887425,0xa2f397ebed9619c64e13afc639d35483055c6efa02ea...,0.0000000989,0.0005,0x092472d7c20c42a9213a4ad4e602c1371eb127a1,1534320629,0.499999999999999999,0x60ca844d8cfbe65cf4f2ad20aa72690c938dcfc32f4d...,buy,349.9999999999999993,ac552320-a062-11e8-8e01-6b262cbc1cee
8,2102453.77813845226339891,4204.907556276904526798,2018-08-15 08:10:29,12032.355915065721774046,0x36789debf58f03ec3d889d01e0b60af01c887425,0x9839ec5f6a84de4f4be02189a1333c5698e7e1426a62...,0.0000000989,0.000207932678657893,0x092472d7c20c42a9213a4ad4e602c1371eb127a1,1534320629,0.207932678657892928,0xfec9d2cd8410149e605ca71a5721f0725dbffbd87ab9...,buy,145.5528750605250496,ac579420-a062-11e8-8e01-6b262cbc1cee
9,2953157.950880759061619258,5906.315901761518123239,2018-08-15 08:09:56,12032.355915065722586859,0x36789debf58f03ec3d889d01e0b60af01c887425,0x9839ec5f6a84de4f4be02189a1333c5698e7e1426a62...,0.0000000989,0.000292067321342107,0x092472d7c20

In [119]:
dag_hist = Analytics.prepare_trade_history_time_line(dag_hist)

In [120]:
dag_hist.query('bid_date == "2018-08-07" and (bid_hour == 0 or bid_hour == 1)')[['price', 'total', 'dt']]

,price,total,dt
425,0.000007,0.374999,2018-08-07 01:47:30
426,0.000007,0.749999,2018-08-07 01:47:30
427,0.000007,1.622674,2018-08-07 01:47:30
428,0.000007,0.974974,2018-08-07 01:47:30
429,0.000008,0.400000,2018-08-07 00:49:51
430,0.000008,0.571477,2018-08-07 00:49:51
431,0.000008,1.566761,2018-08-07 00:49:51
432,0.000008,0.200000,2018-08-07 00:49:51
433,0.000007,0.233690,2018-08-07 00:46:26


In [63]:
dag_hist.query('bid_date == "2018-08-07"')[['price', 'total', 'dt']]

,price,total,dt
dt,,,
2018-08-07 23:21:22,1.300000e-07,0.058500,2018-08-07 23:21:22
2018-08-07 21:31:11,2.100001e-08,0.151000,2018-08-07 21:31:11
2018-08-07 21:31:03,2.320000e-08,0.210000,2018-08-07 21:31:03
2018-08-07 21:30:34,2.330000e-08,0.150100,2018-08-07 21:30:34
2018-08-07 21:29:12,5.010010e-08,0.150000,2018-08-07 21:29:12
2018-08-07 21:29:00,6.100000e-08,0.245017,2018-08-07 21:29:00
2018-08-07 21:28:50,7.000000e-08,0.168362,2018-08-07 21:28:50
2018-08-07 21:27:24,1.200000e-07,0.199345,2018-08-07 21:27:24
2018-08-07 21:27:07,1.220000e-07,0.636594,2018-08-07 21:27:07


In [22]:
0.412/0.0005

823.9999999999999

In [121]:
dag_hist.iloc[1].price

6.1e-06

In [100]:
reload(Analytics)

<module 'Analytics' from 'Analytics.py'>

In [122]:
proc_hist = Analytics.filter_price_out_lier(dag_hist)

3453 2018-07-07 07:40:56 1.526e-05
3454 2018-07-07 07:40:34 1.52652479437e-05
3455 2018-07-07 07:38:46 1e+18
3456 2018-07-07 07:36:53 1.52749151e-05
3457 2018-07-07 07:35:22 1.5273929059e-05
3458 2018-07-07 07:33:36 1.42180559948e-05
3459 2018-07-07 07:32:31 1.528392e-05
3462 2018-07-07 07:25:06 1.5293930328e-05
3463 2018-07-07 07:25:06 1.524e-05
3464 2018-07-07 07:25:06 1.52e-05
3465 2018-07-07 07:21:50 1.40290233347e-05
3466 2018-07-07 07:21:34 1.52e-05
3467 2018-07-07 07:21:13 1.52e-05
3468 2018-07-07 07:18:02 1.52e-05
3471 2018-07-07 07:13:59 1.52e-05
3472 2018-07-07 07:13:02 1.5e-05
3473 2018-07-07 07:09:17 1.52e-05
3474 2018-07-07 07:08:40 1.523e-05
3475 2018-07-07 07:07:40 1.5e-05
3476 2018-07-07 07:06:23 1.524e-05
3477 2018-07-07 07:05:57 1.49e-05
3478 2018-07-07 07:05:33 1.453e-05
3479 2018-07-07 07:04:53 1.49e-05
3481 2018-07-07 07:02:23 1.523e-05
3482 2018-07-07 07:02:05 1.452e-05
3483 2018-07-07 07:01:51 1.524e-05
3484 2018-07-07 07:00:38 1.524e-05
3485 2018-07-07 07:00:15 

3899 2018-07-06 14:48:50 1.045e-05
3900 2018-07-06 14:48:50 1.044e-05
3901 2018-07-06 14:48:50 1.043e-05
3902 2018-07-06 14:48:38 1.02324889541e-05
3903 2018-07-06 14:48:38 1.02238454041e-05
3904 2018-07-06 14:27:36 1.01179383176e-05
3905 2018-07-06 14:24:52 1.044e-05
3906 2018-07-06 14:24:52 1.04301273638e-05
3907 2018-07-06 14:18:46 1.025e-05
3908 2018-07-06 14:16:46 1.025e-05
3909 2018-07-06 14:11:50 1.04319763505e-05
3910 2018-07-06 14:05:30 1.025e-05
3911 2018-07-06 14:02:59 1.04111877573e-05
3912 2018-07-06 14:02:28 1.028e-05
3913 2018-07-06 14:00:39 1.028e-05
3914 2018-07-06 14:00:22 1.021099e-05
3915 2018-07-06 14:00:10 1.020099e-05
3916 2018-07-06 13:57:38 1.0188e-05
3917 2018-07-06 13:14:39 1.00069932206e-05
3918 2018-07-06 13:08:14 9.8085688714e-06
3919 2018-07-06 13:08:14 9.8e-06
3920 2018-07-06 13:08:14 9.7567536015e-06
3921 2018-07-06 13:08:14 9.58e-06
3922 2018-07-06 13:05:25 9.97054232413e-06
3923 2018-07-06 13:05:13 9.97054232413e-06
3924 2018-07-06 13:05:13 9.96093280

4230 2018-07-05 10:04:02 1.009e-05
4231 2018-07-05 10:04:02 1.009e-05
4232 2018-07-05 10:04:02 1e-05
4233 2018-07-05 10:04:02 1e-05
4234 2018-07-05 10:04:02 1e-05
4235 2018-07-05 10:04:02 9.3e-06
4236 2018-07-05 10:04:02 9.3e-06
4237 2018-07-05 10:04:01 1.15089652368e-05
4238 2018-07-05 10:02:48 1.2906e-05
4239 2018-07-05 10:02:36 1.2918e-05
4240 2018-07-05 10:02:16 1.2002e-05
4241 2018-07-05 10:02:16 1.2e-05
4242 2018-07-05 09:57:37 1.3e-05
4243 2018-07-05 09:57:37 1.29e-05
4244 2018-07-05 09:57:37 1.285e-05
4245 2018-07-05 09:57:37 1.25e-05
4246 2018-07-05 09:57:37 1.235e-05
4247 2018-07-05 09:57:37 1.234e-05
4248 2018-07-05 09:57:20 1.136999998e-05
4249 2018-07-05 09:57:20 1.134e-05
4250 2018-07-05 09:57:19 1.15e-05
4251 2018-07-05 09:57:19 1.148e-05
4252 2018-07-05 09:57:19 1.14e-05
4253 2018-07-05 09:57:19 1.13699999998e-05
4254 2018-07-05 09:57:09 1.134e-05
4255 2018-07-05 09:57:09 1.13222e-05
4256 2018-07-05 09:57:09 1.13202e-05
4257 2018-07-05 09:57:00 1.09744774212e-05
4258 20

4552 2018-07-04 22:11:37 1.354e-05
4553 2018-07-04 21:48:36 1.354e-05
4554 2018-07-04 21:41:51 1.354e-05
4555 2018-07-04 21:41:51 1.3539e-05
4556 2018-07-04 21:41:51 1.35e-05
4557 2018-07-04 21:41:51 1.35e-05
4558 2018-07-04 21:28:19 1.35e-05
4559 2018-07-04 21:28:07 1.34999999e-05
4560 2018-07-04 21:15:28 1.34e-05
4561 2018-07-04 20:58:34 1.34e-05
4562 2018-07-04 20:26:42 1.34999999e-05
4563 2018-07-04 20:22:20 1.34999999e-05
4564 2018-07-04 19:33:29 1.346e-05
4565 2018-07-04 19:31:28 1.30789607227e-05
4566 2018-07-04 19:24:07 1.346e-05
4567 2018-07-04 19:13:31 1.346e-05
4568 2018-07-04 19:11:44 1.33753927708e-05
4569 2018-07-04 19:11:35 1.33657737612e-05
4570 2018-07-04 19:11:20 1.32044914618e-05
4571 2018-07-04 18:50:58 1.29730484932e-05
4572 2018-07-04 18:50:27 1.30202544161e-05
4573 2018-07-04 18:50:14 1.3010594968e-05
4574 2018-07-04 18:50:06 1.30846854846e-05
4575 2018-07-04 18:37:51 1.349999e-05
4576 2018-07-04 18:37:32 1.349e-05
4577 2018-07-04 18:36:34 1.339e-05
4578 2018-07-

4908 2018-07-03 21:35:48 1.36671831762e-05
4909 2018-07-03 21:18:14 1.305e-05
4910 2018-07-03 21:16:33 1.305e-05
4911 2018-07-03 21:16:28 1.322e-05
4912 2018-07-03 21:16:21 1.329e-05
4913 2018-07-03 21:13:12 1.33185334063e-05
4914 2018-07-03 21:11:58 1.33185334063e-05
4915 2018-07-03 21:11:10 1.33279591365e-05
4916 2018-07-03 21:08:02 1.37043819013e-05
4917 2018-07-03 21:07:40 1.37041524224e-05
4918 2018-07-03 20:36:56 1.339e-05
4919 2018-07-03 20:36:56 1.339e-05
4920 2018-07-03 20:36:56 1.326e-05
4921 2018-07-03 20:33:54 1.38279641103e-05
4922 2018-07-03 20:26:28 1.38784542065e-05
4923 2018-07-03 20:19:38 1.37974289412e-05
4924 2018-07-03 20:17:01 1.326e-05
4925 2018-07-03 20:16:57 1.3260001e-05
4926 2018-07-03 20:16:52 1.33e-05
4927 2018-07-03 20:16:47 1.3310001e-05
4928 2018-07-03 20:16:38 1.3310001e-05
4929 2018-07-03 20:13:48 1.338e-05
4930 2018-07-03 20:13:23 1.4e-05
4931 2018-07-03 20:13:23 1.33893695643e-05
4932 2018-07-03 20:08:27 1.4e-05
4933 2018-07-03 20:05:00 1.34547421532

In [56]:
print dag_hist.iloc[1].date
print dag_hist.iloc[2].date
print dag_hist.iloc[3].date

2018-08-15 08:58:43
2018-08-15 08:38:17
2018-08-15 08:28:35


In [89]:
print drops

[Timestamp('2018-08-07 03:40:56'), Timestamp('2018-08-07 02:01:12'), Timestamp('2018-08-07 02:01:12'), Timestamp('2018-08-07 00:58:19'), Timestamp('2018-08-07 00:57:27'), Timestamp('2018-08-07 00:48:26'), Timestamp('2018-08-07 00:47:44'), Timestamp('2018-08-07 00:47:41'), Timestamp('2018-08-07 00:47:36'), Timestamp('2018-08-07 00:47:19'), Timestamp('2018-08-07 00:46:56'), Timestamp('2018-08-07 00:46:50')]


In [123]:
proc_hist.query('bid_date == "2018-07-07"')[['price', 'total', 'dt']]

,price,total,dt
3259,1.437955e-05,0.662376,2018-07-07 23:58:07
3260,1.437955e-05,2.539699,2018-07-07 23:52:22
3261,1.430000e-05,0.624739,2018-07-07 23:51:12
3262,1.423472e-05,0.199860,2018-07-07 23:49:25
3263,1.425336e-05,0.382978,2018-07-07 23:49:15
3264,1.420220e-05,4.839479,2018-07-07 23:46:17
3265,1.350000e-05,0.952332,2018-07-07 23:41:34
3266,1.351000e-05,1.648521,2018-07-07 23:38:30
3267,1.335000e-05,11.415609,2018-07-07 23:36:50
3268,1.335010e-05,2.274432,2018-07-07 23:36:37


In [127]:
dag_hist.query('bid_date == "2018-07-07" and price < 1')['price'].mean()

1.3809411674038386e-05

In [125]:
dag_hist.query('bid_date == "2018-07-07"')[['price', 'total', 'dt']]

,price,total,dt
3259,0.000014,0.662376,2018-07-07 23:58:07
3260,0.000014,2.539699,2018-07-07 23:52:22
3261,0.000014,0.624739,2018-07-07 23:51:12
3262,0.000014,0.199860,2018-07-07 23:49:25
3263,0.000014,0.382978,2018-07-07 23:49:15
3264,0.000014,4.839479,2018-07-07 23:46:17
3265,0.000014,0.952332,2018-07-07 23:41:34
3266,0.000014,1.648521,2018-07-07 23:38:30
3267,0.000013,11.415609,2018-07-07 23:36:50
3268,0.000013,2.274432,2018-07-07 23:36:37


In [ ]:
0.213 / 

In [11]:
dt_cols = ['bid_date', 'bid_hour', 'bid_minute', 'dt']

In [12]:
dag_hist[dt_cols]

,bid_date,bid_hour,bid_minute,dt
0,2018-08-02,17,54,2018-08-02 17:54:28
1,2018-08-02,17,51,2018-08-02 17:51:11
2,2018-08-02,17,49,2018-08-02 17:49:43
3,2018-08-02,17,24,2018-08-02 17:24:01
4,2018-08-02,17,24,2018-08-02 17:24:01
5,2018-08-02,17,24,2018-08-02 17:24:01
6,2018-08-02,17,24,2018-08-02 17:24:01
7,2018-08-02,17,24,2018-08-02 17:24:01
8,2018-08-02,17,23,2018-08-02 17:23:53
9,2018-08-02,17,23,2018-08-02 17:23:53


In [12]:
dag_agg = dag_hist.groupby(['bid_date', 'bid_hour', 'bid_minute'])

In [13]:
dag_agg['timestamp'].count().sort_index(level=['bid_date', 'bid_hour', 'bid_minute'], ascending=False)

bid_date    bid_hour  bid_minute
2018-07-08  10        51             1
                      49             2
                      48             1
                      47             1
            8         51             1
                      48             1
                      29             1
                      19             3
                      7              1
            7         53             1
                      35             1
                      29             2
                      28             2
                      26             1
                      18             1
                      17             1
                      9              1
                      2              2
                      1              6
            6         46             1
            5         3              1
                      2              1
            4         59             1
            0         46             2
2018-07-07  23        37       

In [14]:
dag_agg['total'].sum().sort_index(level=['bid_date', 'bid_hour', 'bid_minute'], ascending=False)

bid_date    bid_hour  bid_minute
2018-07-08  10        51             0.075054
                      49             0.892359
                      48             0.116294
                      47             0.116294
            8         51             1.166951
                      48             0.871770
                      29             0.521853
                      19             0.726289
                      7              1.371120
            7         53             1.873492
                      35             4.280507
                      29            11.098566
                      28             5.498946
                      26             7.501615
                      18             1.071003
                      17             7.348463
                      9              2.537018
                      2              1.073032
                      1             32.889597
            6         46             1.547601
            5         3              0.439564
 

## get volume over a price level

In [15]:
dag_hist['dt']

0      2018-07-08 10:51:15
1      2018-07-08 10:49:59
2      2018-07-08 10:49:20
3      2018-07-08 10:48:21
4      2018-07-08 10:47:15
5      2018-07-08 08:51:50
6      2018-07-08 08:48:37
7      2018-07-08 08:29:35
8      2018-07-08 08:19:56
9      2018-07-08 08:19:30
10     2018-07-08 08:19:12
11     2018-07-08 08:07:49
12     2018-07-08 07:53:49
13     2018-07-08 07:35:36
14     2018-07-08 07:29:03
15     2018-07-08 07:29:03
16     2018-07-08 07:28:13
17     2018-07-08 07:28:12
18     2018-07-08 07:26:29
19     2018-07-08 07:18:00
20     2018-07-08 07:17:32
21     2018-07-08 07:09:24
22     2018-07-08 07:02:01
23     2018-07-08 07:02:01
24     2018-07-08 07:01:02
               ...        
4975   2018-06-20 00:29:25
4976   2018-06-20 00:29:25
4977   2018-06-20 00:29:25
4978   2018-06-20 00:29:25
4979   2018-06-20 00:28:52
4980   2018-06-20 00:28:32
4981   2018-06-20 00:28:16
4982   2018-06-20 00:28:15
4983   2018-06-20 00:28:15
4984   2018-06-20 00:28:15
4985   2018-06-20 00:25:42
4

In [19]:
dag_hist[(dag_hist['dt'] >= '2018-07-07 11:00:00') & (dag_hist['price'] >= 0.000060)]['total'].sum()

1.9262890808783042

In [20]:
dag_hist[(dag_hist['dt'] >= '2018-07-07 11:00:00') & (dag_hist['price'] >= 0.000060)]

,amount,buyerFee,date,gasFee,maker,orderHash,price,sellerFee,taker,timestamp,total,transactionHash,type,usdValue,uuid,dt,bid_hour,bid_minute,bid_date
0,1210.54757606,-46.203306910129031905,2018-07-08 03:51:15,167.258064516129031905,0x617b2bda596441ca2428c4bcb6e28ac28b9acdb0,0x9fd10e74a1653bcc9573a05f52ef1a823ebaaa2b217f...,0.000062,0.00007505394971572,0x79b01d97e4a0a81f74db338931ff9ec7a8adc5ad,1531021875,0.075054,0xbf9388fa979e8733b9a327f1b99227ad6e1bc7e3a287...,buy,52.537764801004,29aa6980-8262-11e8-b83e-317d974b7934,2018-07-08 10:51:15,10,51,2018-07-08
1,3000.84496124,6.00168992248,2018-07-08 03:49:59,167.258064516129031652,0x617b2bda596441ca2428c4bcb6e28ac28b9acdb0,0x9fd10e74a1653bcc9573a05f52ef1a823ebaaa2b217f...,0.000062,0.00018605238759688,0x79b01d97e4a0a81f74db338931ff9ec7a8adc5ad,1531021799,0.186052,0x083d871c858dc4c517f2493b2c65340cf921994a9fd4...,buy,130.236671317816,fcbf6380-8261-11e8-b83e-317d974b7934,2018-07-08 10:49:59,10,49,2018-07-08
2,11406.75567259,22.81351134518,2018-07-08 03:49:20,167.474160206718528636,0xa4052c64094c00ef68e9c0135bd8e4669f8b2967,0x08140c6425de15d7c3951f6c39179e25a6c006d88799...,0.000062,0.000706306311246772,0x79b01d97e4a0a81f74db338931ff9ec7a8adc5ad,1531021760,0.706306,0xec28356f4c7cc0bbb2a8f6c31e09a4a6f6059afb3e48...,buy,494.4144178727404,e52054a0-8261-11e8-b83e-317d974b7934,2018-07-08 10:49:20,10,49,2018-07-08
3,1900.84496123,3.80168992246,2018-07-08 03:48:21,169.499864252109968591,0x33f3341d95adefed3a51358b04b599862f0f2b8c,0xb31d61070fbf9b18e8c3f25de4311ac20fbb8a24b987...,0.000061,0.000116293675720214,0x79b01d97e4a0a81f74db338931ff9ec7a8adc5ad,1531021701,0.116294,0x126b76f77c12e28d49379f329879ca85e52252fb2fa9...,buy,81.4055730041495109,c21b7ca0-8261-11e8-b83e-317d974b7934,2018-07-08 10:48:21,10,48,2018-07-08
4,1900.84496123,3.80168992246,2018-07-08 03:47:15,169.499864252109968591,0x33f3341d95adefed3a51358b04b599862f0f2b8c,0xb31d61070fbf9b18e8c3f25de4311ac20fbb8a24b987...,0.000061,0.000116293675720214,0x79b01d97e4a0a81f74db338931ff9ec7a8adc5ad,1531021635,0.116294,0x4c63c8295fded93d0b1e7a7c1307a2798a430611f56f...,buy,81.4055730041495109,9a89f090-8261-11e8-b83e-317d974b7934,2018-07-08 10:47:15,10,47,2018-07-08
8,2000,4,2018-07-08 01:19:56,166.72117795377214,0x33f3341d95adefed3a51358b04b599862f0f2b8c,0xb31d61070fbf9b18e8c3f25de4311ac20fbb8a24b987...,0.000061,0.00012235998,0xa3c3db6f938d3fb511b4daaecb9a4eb95d797c0d,1531012796,0.122360,0x7f37774f1954eaae5276f37e7f41f190f43b3bd25e12...,buy,85.651986,06586870-824d-11e8-b83e-317d974b7934,2018-07-08 08:19:56,8,19,2018-07-08
9,4871.35010777,9.74270021554,2018-07-08 01:19:30,166.721177953974909512,0x1082c5e34252e5e4cbf22489fb815ec3aeee9338,0xd5e0919ee719982902b06c7444a53f64ce0be19f8acf...,0.000061,0.000298029150879505,0xa3c3db6f938d3fb511b4daaecb9a4eb95d797c0d,1531012770,0.298029,0xf9dac9ee7c73ea0bf67e170d637708ba8251944bf282...,buy,208.6204056156535364,f6a90ec0-824c-11e8-b83e-317d974b7934,2018-07-08 08:19:30,8,19,2018-07-08
10,5000,10,2018-07-08 01:19:12,166.721177954317155,0x1082c5e34252e5e4cbf22489fb815ec3aeee9338,0xd5e0919ee719982902b06c7444a53f64ce0be19f8acf...,0.000061,0.000305899949999,0xa3c3db6f938d3fb511b4daaecb9a4eb95d797c0d,1531012752,0.305900,0x9c21ca5ce62c0edab224a518ab2186836f5e285524fa...,buy,214.1299649993,ec633490-824c-11e8-b83e-317d974b7934,2018-07-08 08:19:12,8,19,2018-07-08


## Hourly Volume

In [47]:
dag_dt = dag_hist
dag_dt.index = dag_dt['dt']

In [50]:
hourly_vol = dag_dt['total'].resample('1H').sum()

In [51]:
hourly_vol = hourly_vol.fillna(0)

In [14]:
hourly_vol[9]

1.5072210727453503

In [15]:
hourly_vol

dt
2018-06-08 13:00:00    0.346220
2018-06-08 14:00:00    0.000000
2018-06-08 15:00:00    0.287250
2018-06-08 16:00:00    0.000000
2018-06-08 17:00:00    0.000000
2018-06-08 18:00:00    0.000000
2018-06-08 19:00:00    0.000000
2018-06-08 20:00:00    2.351815
2018-06-08 21:00:00    1.653282
2018-06-08 22:00:00    1.507221
2018-06-08 23:00:00    0.151000
2018-06-09 00:00:00    0.000000
2018-06-09 01:00:00    0.000000
2018-06-09 02:00:00    0.000000
2018-06-09 03:00:00    0.000000
2018-06-09 04:00:00    0.000000
2018-06-09 05:00:00    0.000000
2018-06-09 06:00:00    0.000000
2018-06-09 07:00:00    0.000000
2018-06-09 08:00:00    0.000000
2018-06-09 09:00:00    0.000000
2018-06-09 10:00:00    2.716291
2018-06-09 11:00:00    0.000000
2018-06-09 12:00:00    0.000000
2018-06-09 13:00:00    0.710626
                         ...   
2018-08-12 13:00:00    0.000000
2018-08-12 14:00:00    3.802212
2018-08-12 15:00:00    0.000000
2018-08-12 16:00:00    0.000000
2018-08-12 17:00:00    0.646600
2018-

In [48]:
hourly_max = dag_dt['price'].resample('1H').max().fillna(0)

In [52]:
print hourly_max.index[1], hourly_max[1]

2018-08-07 01:00:00 1.1e-06


In [53]:
print hourly_max.index[2], hourly_max[2]

2018-08-07 02:00:00 1e-07


In [55]:
print hourly_max.index[3], hourly_max[3]

2018-08-07 03:00:00 1.1e-06


In [50]:
hourly_max[10:40].max()

2.5000000000000002e-06

In [54]:
hourly_max

dt
2018-08-07 00:00:00    4.120000e-01
2018-08-07 01:00:00    1.100000e-06
2018-08-07 02:00:00    1.000000e-07
2018-08-07 03:00:00    1.100000e-06
2018-08-07 04:00:00    0.000000e+00
2018-08-07 05:00:00    1.100000e-06
2018-08-07 06:00:00    0.000000e+00
2018-08-07 07:00:00    0.000000e+00
2018-08-07 08:00:00    0.000000e+00
2018-08-07 09:00:00    0.000000e+00
2018-08-07 10:00:00    0.000000e+00
2018-08-07 11:00:00    0.000000e+00
2018-08-07 12:00:00    1.210000e-07
2018-08-07 13:00:00    0.000000e+00
2018-08-07 14:00:00    0.000000e+00
2018-08-07 15:00:00    0.000000e+00
2018-08-07 16:00:00    0.000000e+00
2018-08-07 17:00:00    3.200000e-07
2018-08-07 18:00:00    0.000000e+00
2018-08-07 19:00:00    3.900000e-07
2018-08-07 20:00:00    0.000000e+00
2018-08-07 21:00:00    1.233000e-07
2018-08-07 22:00:00    0.000000e+00
2018-08-07 23:00:00    1.300000e-07
2018-08-08 00:00:00    0.000000e+00
                           ...     
2018-08-14 16:00:00    8.800000e-08
2018-08-14 17:00:00    9.

In [52]:
## don't fill 0 to this
hourly_min = dag_dt['price'].resample('1H').min()

In [53]:
hourly_min

dt
2018-06-08 13:00:00    0.000002
2018-06-08 14:00:00         NaN
2018-06-08 15:00:00    0.000002
2018-06-08 16:00:00         NaN
2018-06-08 17:00:00         NaN
2018-06-08 18:00:00         NaN
2018-06-08 19:00:00         NaN
2018-06-08 20:00:00    0.000002
2018-06-08 21:00:00    0.000002
2018-06-08 22:00:00    0.000002
2018-06-08 23:00:00    0.000002
2018-06-09 00:00:00         NaN
2018-06-09 01:00:00         NaN
2018-06-09 02:00:00         NaN
2018-06-09 03:00:00         NaN
2018-06-09 04:00:00         NaN
2018-06-09 05:00:00         NaN
2018-06-09 06:00:00         NaN
2018-06-09 07:00:00         NaN
2018-06-09 08:00:00         NaN
2018-06-09 09:00:00         NaN
2018-06-09 10:00:00    0.000002
2018-06-09 11:00:00         NaN
2018-06-09 12:00:00         NaN
2018-06-09 13:00:00    0.000002
                         ...   
2018-08-13 18:00:00         NaN
2018-08-13 19:00:00         NaN
2018-08-13 20:00:00         NaN
2018-08-13 21:00:00         NaN
2018-08-13 22:00:00         NaN
2018-

In [54]:
hourly_min_ff = hourly_min.fillna(method='ffill')

In [56]:
hourly_min_ff.loc['2018-08-09 18:00:00':'2018-08-10 18:00:00']

dt
2018-08-09 18:00:00    0.000010
2018-08-09 19:00:00    0.000010
2018-08-09 20:00:00    0.000010
2018-08-09 21:00:00    0.000010
2018-08-09 22:00:00    0.000010
2018-08-09 23:00:00    0.000012
2018-08-10 00:00:00    0.000012
2018-08-10 01:00:00    0.000012
2018-08-10 02:00:00    0.000012
2018-08-10 03:00:00    0.000011
2018-08-10 04:00:00    0.000011
2018-08-10 05:00:00    0.000011
2018-08-10 06:00:00    0.000011
2018-08-10 07:00:00    0.000010
2018-08-10 08:00:00    0.000010
2018-08-10 09:00:00    0.000012
2018-08-10 10:00:00    0.000012
2018-08-10 11:00:00    0.000012
2018-08-10 12:00:00    0.000012
2018-08-10 13:00:00    0.000011
2018-08-10 14:00:00    0.000012
2018-08-10 15:00:00    0.000012
2018-08-10 16:00:00    0.000012
2018-08-10 17:00:00    0.000012
2018-08-10 18:00:00    0.000012
Freq: H, Name: price, dtype: float64

In [72]:
len(hourly_min_ff)

1614

In [76]:
hourly_min_ff[1612:1734]

dt
2018-08-14 17:00:00    0.00001
2018-08-14 18:00:00    0.00001
Freq: H, Name: price, dtype: float64

## Find the 0 gaps
Continuously search for 10 interval with trade volume less than 0.5 EHT?

In [29]:
for i in range(len(hourly_vol)-1, 10, -3):
    if hourly_vol[i] > 10*hourly_vol[(i-10):i].sum():
        print "{}: {}".format(hourly_vol.index[i], hourly_vol[i])

2018-08-13 13:00:00: 0.16150105
2018-08-04 07:00:00: 32.3700268742
2018-08-03 13:00:00: 7.52532809556
2018-08-02 07:00:00: 17.1348906096
2018-07-10 13:00:00: 0.313961234206
2018-07-08 13:00:00: 0.431016370426
2018-06-19 07:00:00: 0.49752076
2018-06-09 10:00:00: 2.71629111502


In [36]:
for i in range(len(hourly_vol)-1, 10, -1):
    if hourly_vol[i] > 10*hourly_vol[(i-10):i].sum():
        print "{}: {}".format(hourly_vol.index[i], hourly_vol[i])

2018-08-13 13:00:00: 0.16150105
2018-08-12 14:00:00: 3.80221160842
2018-08-12 12:00:00: 0.065199
2018-08-04 07:00:00: 32.3700268742
2018-08-03 13:00:00: 7.52532809556
2018-08-02 07:00:00: 17.1348906096
2018-07-25 14:00:00: 1.40793902597
2018-07-15 18:00:00: 0.18
2018-07-11 17:00:00: 1.98627
2018-07-10 13:00:00: 0.313961234206
2018-07-09 18:00:00: 0.32336
2018-07-08 13:00:00: 0.431016370426
2018-07-07 15:00:00: 0.1638
2018-07-06 20:00:00: 2.50456031503
2018-07-04 15:00:00: 2.21583557327
2018-07-03 14:00:00: 0.942094932393
2018-06-30 06:00:00: 0.766199162099
2018-06-27 17:00:00: 1.00858910267
2018-06-26 12:00:00: 2.53537690083
2018-06-24 14:00:00: 0.3224
2018-06-22 17:00:00: 0.668619354839
2018-06-21 09:00:00: 0.0805780887103
2018-06-19 07:00:00: 0.49752076
2018-06-16 12:00:00: 0.652142563674
2018-06-09 10:00:00: 2.71629111502


In [69]:
def find_0_gaps_max_incr(hourly_vol, hourly_max, adj_look_back=10, adj_vol_incr=10, adj_max_incr=1.3, adj_next = 24):
    for i in range(len(hourly_vol)-1, 10, -1):
        last_10 = hourly_vol[(i-adj_look_back):i].sum()
        next_max = hourly_max[i+1:(i+adj_next)].max()
        if hourly_vol[i] > adj_vol_incr*last_10 and next_max > (adj_max_incr*hourly_max[i]):
#         if hourly_vol[i] > 5*last_10:
            print "{}: {}".format(hourly_vol.index[i], hourly_vol[i])
            print 'beat: ',last_10 
            incr_price = 100*(next_max/hourly_max[i])
            print "next: {} | now: {} | incr: {}".format(next_max, hourly_max[i], incr_price)

In [42]:
for i in range(len(hourly_vol)-1, 10, -1):
    last_10 = hourly_vol[(i-10):i].sum()
    if hourly_vol[i] > 5*last_10:
        print "{}: {}".format(hourly_vol.index[i], hourly_vol[i])
        print 'beat: ',last_10 

2018-08-13 13:00:00: 0.16150105
beat:  0.0
2018-08-12 14:00:00: 3.80221160842
beat:  0.065199
2018-08-12 12:00:00: 0.065199
beat:  0.0
2018-08-09 12:00:00: 14.9244204456
beat:  2.02461684138
2018-08-07 13:00:00: 755.866484216
beat:  95.1780216889
2018-08-04 07:00:00: 32.3700268742
beat:  0.10716
2018-08-03 13:00:00: 7.52532809556
beat:  0.6523965
2018-08-02 07:00:00: 17.1348906096
beat:  1.47441032601
2018-07-25 14:00:00: 1.40793902597
beat:  0.0588139
2018-07-18 12:00:00: 89.7057678688
beat:  17.7903600504
2018-07-16 15:00:00: 1.49947735
beat:  0.261901244734
2018-07-15 19:00:00: 0.948916588018
beat:  0.18
2018-07-15 18:00:00: 0.18
beat:  0.0
2018-07-13 00:00:00: 0.870573873443
beat:  0.136307903751
2018-07-12 05:00:00: 1.65356971784
beat:  0.212
2018-07-11 17:00:00: 1.98627
beat:  0.0
2018-07-10 13:00:00: 0.313961234206
beat:  0.0
2018-07-09 18:00:00: 0.32336
beat:  0.0
2018-07-08 13:00:00: 0.431016370426
beat:  0.0
2018-07-07 17:00:00: 1.3675661036
beat:  0.1638
2018-07-07 15:00:00:

In [68]:
find_0_gaps_max_incr(hourly_vol, hourly_max)

2018-08-02 07:00:00: 17.1348906096
beat:  1.47441032601
next: 1.68e-05 | now: 1.285e-05 | incr: 130.739299611
2018-07-25 14:00:00: 1.40793902597
beat:  0.0588139
next: 1.348e-05 | now: 6.86e-06 | incr: 196.501457726


In [70]:
find_0_gaps_max_incr(hourly_vol, hourly_max, adj_look_back=3, adj_vol_incr=10, adj_max_incr=1.3, adj_next = 24)

2018-08-05 07:00:00: 1.917448472
beat:  0.0
next: 1.96666e-05 | now: 1.159e-05 | incr: 169.685936152
2018-08-02 07:00:00: 17.1348906096
beat:  0.0
next: 1.68e-05 | now: 1.285e-05 | incr: 130.739299611
2018-07-25 23:00:00: 1.45550654023
beat:  0.0
next: 1.348e-05 | now: 7.83939953958e-06 | incr: 171.951945196
2018-07-25 14:00:00: 1.40793902597
beat:  0.0
next: 1.348e-05 | now: 6.86e-06 | incr: 196.501457726
2018-07-25 10:00:00: 0.0588139
beat:  0.0
next: 1.348e-05 | now: 7.01e-06 | incr: 192.296718973
2018-07-16 21:00:00: 0.972436943985
beat:  0.0
next: 4.95e-06 | now: 2.58e-06 | incr: 191.860465116
2018-07-16 15:00:00: 1.49947735
beat:  0.0
next: 3.46e-06 | now: 2.43e-06 | incr: 142.386831276
2018-07-13 16:00:00: 0.262691068161
beat:  0.0
next: 2.5e-06 | now: 1.8205e-06 | incr: 137.324910739
2018-07-13 00:00:00: 0.870573873443
beat:  0.0
next: 2.5e-06 | now: 1.9e-06 | incr: 131.578947368
2018-07-02 19:00:00: 1.6173936616
beat:  0.0
next: 2.46e-06 | now: 1.58e-06 | incr: 155.696202532
2

In [71]:
find_0_gaps_max_incr(hourly_vol, hourly_max, adj_look_back=3, adj_vol_incr=10, adj_max_incr=1.6, adj_next = 24)

2018-08-05 07:00:00: 1.917448472
beat:  0.0
next: 1.96666e-05 | now: 1.159e-05 | incr: 169.685936152
2018-07-25 23:00:00: 1.45550654023
beat:  0.0
next: 1.348e-05 | now: 7.83939953958e-06 | incr: 171.951945196
2018-07-25 14:00:00: 1.40793902597
beat:  0.0
next: 1.348e-05 | now: 6.86e-06 | incr: 196.501457726
2018-07-25 10:00:00: 0.0588139
beat:  0.0
next: 1.348e-05 | now: 7.01e-06 | incr: 192.296718973
2018-07-16 21:00:00: 0.972436943985
beat:  0.0
next: 4.95e-06 | now: 2.58e-06 | incr: 191.860465116


In [34]:
def find_0_gaps_max_incr_analysis(hourly_vol, hourly_max, hourly_min, adj_look_back=10, adj_vol_incr=10, price_max_incr=30, adj_next=24):
    result = {}
    
#     print 'init result: ', result
#     print "arg adj_look_back={}, adj_vol_incr={}, price_max_incr={}, adj_next = {}".format(adj_look_back, adj_vol_incr, price_max_incr, adj_next)
    
    steps = 0
    incr_steps = 0
    decr_steps = 0
    incr_total = 0
    decr_total = 0
    for i in range(len(hourly_vol)-1, 10, -1):
        steps += 1
        
        last_10 = hourly_vol[(i-adj_look_back):i].sum()
        next_max = hourly_max[i+1:(i+adj_next)].max()
        next_min = hourly_min[i+1:(i+adj_next)].max()
        if hourly_vol[i] > adj_vol_incr*last_10:

#             print "{}: {}".format(hourly_vol.index[i], hourly_vol[i])
#             print 'beat: ',last_10 
            if hourly_max[i] == 0 or next_max == 0:
                continue
            
            incr_price = 100*(next_max/hourly_max[i]) - 100
            if incr_price > price_max_incr:
#                 print "{} next: {} | now: {} | incr: {}".format(hourly_vol.index[i], next_max, hourly_max[i], incr_price)
                incr_steps += 1
                incr_total += incr_price
                continue
            
#             if hourly_min[i] == 0:
#                 continue
                
            desc_price = 100*(next_min/hourly_max[i]) - 100
            if desc_price < -0.5*price_max_incr:
#                 print "{} next: {} | now: {} | descr: {}".format(hourly_vol.index[i], next_max, hourly_max[i], incr_price)
                decr_steps += 1
                decr_total += desc_price
        
#     print '****** Final result *******'
#     print "arg adj_look_back={}, adj_vol_incr={}, price_max_incr={}, adj_next = {}".format(adj_look_back, adj_vol_incr, price_max_incr, adj_next)
#     print steps, incr_steps, decr_steps
#     print "Incresed steps: {}".format(incr_steps/steps)
#     print "Decrease steps: {}".format(decr_steps/steps)
    
    if decr_steps == 0:
        decr_steps = 0.5
    if steps == 0:
        in_ratio = 0
        de_ratio = 0
    else:
        in_ratio = incr_steps/steps
        de_ratio = decr_steps/steps
    
    if incr_steps == 0 or decr_steps == 0:
        incr_avg = 0
        decr_avg = 0
    else:
        incr_avg = incr_total/incr_steps
        decr_avg = decr_total/decr_steps
    
    result['adj_look_back'] = adj_look_back
    result['adj_vol_incr'] = adj_vol_incr
    result['price_max_incr'] = price_max_incr
    result['adj_next'] = adj_next
    result['steps'] = steps
    result['increase'] = in_ratio
    result['decrease'] = de_ratio
    result['incr_total'] = incr_total
    result['decr_total'] = decr_total
    result['incr_avg'] = incr_avg
    result['decr_avg'] = decr_avg
#     print 'return result: ', result
    
    return result
        

In [129]:
find_0_gaps_max_incr_analysis(hourly_vol, hourly_max, hourly_min, adj_look_back=3, adj_vol_incr=10, price_max_incr=30, adj_next = 24)

{'adj_look_back': 3,
 'adj_next': 24,
 'adj_vol_incr': 10,
 'decrease': 0.0006353240152477764,
 'increase': 0.010800508259212199,
 'price_max_incr': 30,
 'steps': 1574}

In [9]:
import itertools

In [101]:
adj_look_backs = [2, 3, 4, 5, 6, 7, 8, 9]
adj_vol_incrs = [2, 3, 4, 5, 6, 7, 8, 9, 10]
price_max_incrs = [10, 20, 25, 30, 35, 40, 45, 50]
adj_nexts = [10, 12, 14, 16, 18, 20, 22, 24]

args_array = [adj_look_backs, adj_vol_incrs, price_max_incrs, adj_nexts]
args = list(itertools.product(*args_array))

In [11]:
adj_look_backs = [2, 4]
adj_vol_incrs = [ 3, 9]
price_max_incrs = [30, 50]
adj_nexts = [10, 24]

args_array = [adj_look_backs, adj_vol_incrs, price_max_incrs, adj_nexts]
args = list(itertools.product(*args_array))

In [181]:
len(list(itertools.product(*args_array)))

192

In [157]:
agrs[0][3]

10

In [180]:
adj_look_backs = [2, 4, 6, 10]
adj_vol_incrs = [ 3, 6, 9]
price_max_incrs = [10, 25, 40, 50]
adj_nexts = [5, 10, 16, 24]

args_array = [adj_look_backs, adj_vol_incrs, price_max_incrs, adj_nexts]
args = list(itertools.product(*args_array))

In [182]:
result = []

for arg in args:
    print arg
    rlt = find_0_gaps_max_incr_analysis(hourly_vol, hourly_max, hourly_min, adj_look_back=arg[0], adj_vol_incr =arg[1], price_max_incr =arg[2], adj_next =arg[3])
    result.append(rlt)
#     print rlt

(2, 3, 10, 5)
(2, 3, 10, 10)
(2, 3, 10, 16)
(2, 3, 10, 24)
(2, 3, 25, 5)
(2, 3, 25, 10)
(2, 3, 25, 16)
(2, 3, 25, 24)
(2, 3, 40, 5)
(2, 3, 40, 10)
(2, 3, 40, 16)
(2, 3, 40, 24)
(2, 3, 50, 5)
(2, 3, 50, 10)
(2, 3, 50, 16)
(2, 3, 50, 24)
(2, 6, 10, 5)
(2, 6, 10, 10)
(2, 6, 10, 16)
(2, 6, 10, 24)
(2, 6, 25, 5)
(2, 6, 25, 10)
(2, 6, 25, 16)
(2, 6, 25, 24)
(2, 6, 40, 5)
(2, 6, 40, 10)
(2, 6, 40, 16)
(2, 6, 40, 24)
(2, 6, 50, 5)
(2, 6, 50, 10)
(2, 6, 50, 16)
(2, 6, 50, 24)
(2, 9, 10, 5)
(2, 9, 10, 10)
(2, 9, 10, 16)
(2, 9, 10, 24)
(2, 9, 25, 5)
(2, 9, 25, 10)
(2, 9, 25, 16)
(2, 9, 25, 24)
(2, 9, 40, 5)
(2, 9, 40, 10)
(2, 9, 40, 16)
(2, 9, 40, 24)
(2, 9, 50, 5)
(2, 9, 50, 10)
(2, 9, 50, 16)
(2, 9, 50, 24)
(4, 3, 10, 5)
(4, 3, 10, 10)
(4, 3, 10, 16)
(4, 3, 10, 24)
(4, 3, 25, 5)
(4, 3, 25, 10)
(4, 3, 25, 16)
(4, 3, 25, 24)
(4, 3, 40, 5)
(4, 3, 40, 10)
(4, 3, 40, 16)
(4, 3, 40, 24)
(4, 3, 50, 5)
(4, 3, 50, 10)
(4, 3, 50, 16)
(4, 3, 50, 24)
(4, 6, 10, 5)
(4, 6, 10, 10)
(4, 6, 10, 16)
(4, 6, 10, 2

In [173]:
result

[{'adj_look_back': 2,
  'adj_next': 10,
  'adj_vol_incr': 3,
  'decrease': 0.0038119440914866584,
  'increase': 0.01397712833545108,
  'price_max_incr': 30,
  'steps': 1574},
 {'adj_look_back': 2,
  'adj_next': 24,
  'adj_vol_incr': 3,
  'decrease': 0.0025412960609911056,
  'increase': 0.02795425667090216,
  'price_max_incr': 30,
  'steps': 1574},
 {'adj_look_back': 2,
  'adj_next': 10,
  'adj_vol_incr': 3,
  'decrease': 0.0012706480304955528,
  'increase': 0.0044472681067344345,
  'price_max_incr': 50,
  'steps': 1574},
 {'adj_look_back': 2,
  'adj_next': 24,
  'adj_vol_incr': 3,
  'decrease': 0.0012706480304955528,
  'increase': 0.014612452350698857,
  'price_max_incr': 50,
  'steps': 1574},
 {'adj_look_back': 2,
  'adj_next': 10,
  'adj_vol_incr': 9,
  'decrease': 0.0025412960609911056,
  'increase': 0.011435832274459974,
  'price_max_incr': 30,
  'steps': 1574},
 {'adj_look_back': 2,
  'adj_next': 24,
  'adj_vol_incr': 9,
  'decrease': 0.0012706480304955528,
  'increase': 0.0216010

In [183]:
rpd = pd.DataFrame(result)

In [184]:
rpd['id_ratio'] = rpd['increase']/rpd['decrease']


In [185]:
rpd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 8 columns):
adj_look_back     192 non-null int64
adj_next          192 non-null int64
adj_vol_incr      192 non-null int64
decrease          192 non-null float64
increase          192 non-null float64
price_max_incr    192 non-null int64
steps             192 non-null int64
id_ratio          192 non-null float64
dtypes: float64(3), int64(5)
memory usage: 12.1 KB


In [186]:
rpd.sort_values('id_ratio', ascending=False)

,adj_look_back,adj_next,adj_vol_incr,decrease,increase,price_max_incr,steps,id_ratio
47,2,24,9,0.000635,0.010165,50,1574,16.000000
15,2,24,3,0.001271,0.014612,50,1574,11.500000
75,4,24,6,0.000635,0.006989,40,1574,11.000000
31,2,24,6,0.001271,0.013342,50,1574,10.500000
79,4,24,6,0.000635,0.006353,50,1574,10.000000
46,2,16,9,0.000635,0.005718,50,1574,9.000000
43,2,24,9,0.001271,0.011436,40,1574,9.000000
11,2,24,3,0.001906,0.015883,40,1574,8.333333
91,4,24,9,0.000635,0.005083,40,1574,8.000000
39,2,24,9,0.003177,0.024778,25,1574,7.800000


In [40]:
print dag_hist.query('bid_date == "2018-08-02" and bid_hour == 17 and price >= 0.00001437')['total'].sum()
print dag_hist.query('bid_date == "2018-08-02" and bid_hour == 17 and price >= 0.000016')['total'].sum()

84.9055851508
40.7591


In [ ]:
adj_look_backs = [2, 4, 6, 10]
adj_vol_incrs = [ 3, 6, 9]
price_max_incrs = [10, 25, 40, 50]
adj_nexts = [5, 10, 16, 24]

args_array = [adj_look_backs, adj_vol_incrs, price_max_incrs, adj_nexts]
args = list(itertools.product(*args_array))

In [204]:
def full_volume_analysis(base_obj, coin, args):
#     base_obj = Base()
    # r = base_obj.request_ticker()
    # print r

    dag_hist = base_obj.get_trade_history(coin)
    dag_hist['total'] = pd.to_numeric(dag_hist['total'])
    dag_hist['price'] = pd.to_numeric(dag_hist['price'])
    dag_hist['dt'] = pd.to_datetime(dag_hist.date, format="%Y-%m-%d %H:%M:%S")

    # convert from UTC to ICT
    dag_hist['dt'] +=  pd.to_timedelta(7, unit='h')

    dag_hist['bid_hour'] = dag_hist['dt'].dt.hour
    dag_hist['bid_minute'] = dag_hist['dt'].dt.minute
    dag_hist['bid_date'] = dag_hist['dt'].dt.strftime('%Y-%m-%d')

    dag_dt = dag_hist
    dag_dt.index = dag_dt['dt']


    hourly_vol = dag_dt['total'].resample('1H').sum().fillna(0)
    hourly_max = dag_dt['price'].resample('1H').max().fillna(0)
    hourly_min = dag_dt['price'].resample('1H').min()
    
    result = []

    for arg in args:
#         print arg
        rlt = find_0_gaps_max_incr_analysis(hourly_vol, hourly_max, hourly_min, adj_look_back=arg[0], adj_vol_incr =arg[1], price_max_incr =arg[2], adj_next =arg[3])
        result.append(rlt)
        
    rpd = pd.DataFrame(result)
    rpd['id_ratio'] = rpd['increase']/rpd['decrease']
    print "********* {} *********".format(coin.upper())
    print rpd.sort_values('id_ratio', ascending=False).head(8)

In [201]:
base_obj = Base()
coins = ['MVL', 'DAG', 'PKG', 'MTC', 'SNTR', 'PCL', 'MET']

for coin in coins:
    full_volume_analysis(base_obj, coin, args)

(2, 3, 20, 10)
(2, 3, 20, 24)
(2, 3, 30, 10)
(2, 3, 30, 24)
(2, 3, 50, 10)
(2, 3, 50, 24)
(2, 3, 60, 10)
(2, 3, 60, 24)
(2, 9, 20, 10)
(2, 9, 20, 24)
(2, 9, 30, 10)
(2, 9, 30, 24)
(2, 9, 50, 10)
(2, 9, 50, 24)
(2, 9, 60, 10)
(2, 9, 60, 24)
(4, 3, 20, 10)
(4, 3, 20, 24)
(4, 3, 30, 10)
(4, 3, 30, 24)
(4, 3, 50, 10)
(4, 3, 50, 24)
(4, 3, 60, 10)
(4, 3, 60, 24)
(4, 9, 20, 10)
(4, 9, 20, 24)
(4, 9, 30, 10)
(4, 9, 30, 24)
(4, 9, 50, 10)
(4, 9, 50, 24)
(4, 9, 60, 10)
(4, 9, 60, 24)
(6, 3, 20, 10)
(6, 3, 20, 24)
(6, 3, 30, 10)
(6, 3, 30, 24)
(6, 3, 50, 10)
(6, 3, 50, 24)
(6, 3, 60, 10)
(6, 3, 60, 24)
(6, 9, 20, 10)
(6, 9, 20, 24)
(6, 9, 30, 10)
(6, 9, 30, 24)
(6, 9, 50, 10)
(6, 9, 50, 24)
(6, 9, 60, 10)
(6, 9, 60, 24)
(8, 3, 20, 10)
(8, 3, 20, 24)
(8, 3, 30, 10)
(8, 3, 30, 24)
(8, 3, 50, 10)
(8, 3, 50, 24)
(8, 3, 60, 10)
(8, 3, 60, 24)
(8, 9, 20, 10)
(8, 9, 20, 24)
(8, 9, 30, 10)
(8, 9, 30, 24)
(8, 9, 50, 10)
(8, 9, 50, 24)
(8, 9, 60, 10)
(8, 9, 60, 24)
********* MVL *********
    adj_look_bac

(8, 9, 20, 24)
(8, 9, 30, 10)
(8, 9, 30, 24)
(8, 9, 50, 10)
(8, 9, 50, 24)
(8, 9, 60, 10)
(8, 9, 60, 24)
********* SNTR *********
    adj_look_back  adj_next  adj_vol_incr  decrease  increase  price_max_incr  steps  id_ratio
1               2        24             3       0.0  0.010695              20    374       inf
3               2        24             3       0.0  0.002674              30    374       inf
9               2        24             9       0.0  0.005348              20    374       inf
11              2        24             9       0.0  0.002674              30    374       inf
17              4        24             3       0.0  0.002674              20    374       inf
0               2        10             3       0.0  0.000000              20    374       NaN
2               2        10             3       0.0  0.000000              30    374       NaN
4               2        10             3       0.0  0.000000              50    374       NaN
(2, 3, 20, 10)


In [205]:
coins = ['BWX', 'CARD', 'UPP', 'ZCN', 'QNT', 'WEB', 'ESS', 'TOMO', 'TGAME']

for coin in coins:
    full_volume_analysis(base_obj, coin, args)

********* BWX *********
    adj_look_back  adj_next  adj_vol_incr  decrease  increase  price_max_incr  steps  id_ratio
8               2        10             9   0.00000  0.001560              20    641       inf
9               2        24             9   0.00000  0.009360              20    641       inf
17              4        24             3   0.00000  0.003120              20    641       inf
25              4        24             9   0.00000  0.001560              20    641       inf
33              6        24             3   0.00000  0.001560              20    641       inf
49              8        24             3   0.00000  0.001560              20    641       inf
1               2        24             3   0.00156  0.014041              20    641  9.000000
0               2        10             3   0.00312  0.001560              20    641  0.500000
********* CARD *********
   adj_look_back  adj_next  adj_vol_incr  decrease  increase  price_max_incr  steps  id_ratio
2 

In [208]:
adj_look_backs = [2, 4, 6, 8]
adj_vol_incrs = [ 6, 12, 15]
price_max_incrs = [20, 30, 60]
adj_nexts = [10, 24]

args_array = [adj_look_backs, adj_vol_incrs, price_max_incrs, adj_nexts]
args = list(itertools.product(*args_array))

In [209]:
coins = ['MVL', 'DAG', 'PKG', 'MTC', 'SNTR', 'PCL', 'MET', 'BWX', 'CARD', 'UPP', 'ZCN', 'QNT', 'WEB', 'ESS', 'TOMO', 'TGAME']

for coin in coins:
    full_volume_analysis(base_obj, coin, args)

********* MVL *********
    adj_look_back  adj_next  adj_vol_incr  decrease  increase  price_max_incr  steps  id_ratio
15              2        24            15  0.003177  0.020330              30   1574  6.400000
9               2        24            12  0.003177  0.020330              30   1574  6.400000
17              2        24            15  0.001271  0.006989              60   1574  5.500000
11              2        24            12  0.001271  0.006989              60   1574  5.500000
3               2        24             6  0.005083  0.025413              30   1574  5.000000
69              8        24            15  0.000635  0.003177              30   1574  5.000000
13              2        24            15  0.005718  0.027319              20   1574  4.777778
27              4        24            12  0.001906  0.008895              30   1574  4.666667
********* DAG *********
    adj_look_back  adj_next  adj_vol_incr  decrease  increase  price_max_incr  steps  id_ratio
67

********* ZCN *********
    adj_look_back  adj_next  adj_vol_incr  decrease  increase  price_max_incr  steps  id_ratio
33              4        24            15       0.0  0.001808              30   1106       inf
31              4        24            15       0.0  0.001808              20   1106       inf
30              4        10            15       0.0  0.000904              20   1106       inf
15              2        24            15       0.0  0.004521              30   1106       inf
14              2        10            15       0.0  0.000904              30   1106       inf
12              2        10            15       0.0  0.002712              20   1106       inf
13              2        24            15       0.0  0.009042              20   1106       inf
32              4        10            15       0.0  0.000904              30   1106       inf
********* QNT *********
    adj_look_back  adj_next  adj_vol_incr  decrease  increase  price_max_incr  steps  id_ratio
27

In [35]:
def back_test_time_segment_volume_analysis(base_obj, coin, args):
#     base_obj = Base()
    # r = base_obj.request_ticker()
    # print r

    dag_hist = base_obj.get_trade_history(coin)
    dag_hist['total'] = pd.to_numeric(dag_hist['total'])
    dag_hist['price'] = pd.to_numeric(dag_hist['price'])
    dag_hist['dt'] = pd.to_datetime(dag_hist.date, format="%Y-%m-%d %H:%M:%S")

    # convert from UTC to ICT
    dag_hist['dt'] +=  pd.to_timedelta(7, unit='h')

    dag_hist['bid_hour'] = dag_hist['dt'].dt.hour
    dag_hist['bid_minute'] = dag_hist['dt'].dt.minute
    dag_hist['bid_date'] = dag_hist['dt'].dt.strftime('%Y-%m-%d')

    dag_dt = dag_hist
    dag_dt.index = dag_dt['dt']

    seg1 = int(len(dag_dt)/3)
    seg2 = 2*seg1
    print seg1, seg2
    
    hourly_vols = []
    hourly_maxs = []
    hourly_mins = []
    
    hourly_vols.append(dag_dt.iloc[:seg1]['total'].resample('1H').sum().fillna(0))
    hourly_vols.append(dag_dt.iloc[seg1:seg2]['total'].resample('1H').sum().fillna(0))
    hourly_vols.append(dag_dt.iloc[seg2:]['total'].resample('1H').sum().fillna(0))
    hourly_vols.append(dag_dt['total'].resample('1H').sum().fillna(0))
    
    hourly_maxs.append(dag_dt.iloc[:seg1]['price'].resample('1H').max().fillna(0))
    hourly_maxs.append(dag_dt.iloc[seg1:seg2]['price'].resample('1H').max().fillna(0))
    hourly_maxs.append(dag_dt.iloc[seg2:]['price'].resample('1H').max().fillna(0))
    hourly_maxs.append(dag_dt['price'].resample('1H').max().fillna(0))
    
    hourly_mins.append(dag_dt.iloc[:seg1]['price'].resample('1H').min().fillna(0))
    hourly_mins.append(dag_dt.iloc[seg1:seg2]['price'].resample('1H').min().fillna(0))
    hourly_mins.append(dag_dt.iloc[seg2:]['price'].resample('1H').min().fillna(0))
    hourly_mins.append(dag_dt['price'].resample('1H').min().fillna(0))
    
#     hourly_mins = dag_dt['price'].resample('1H').min()

    for time_seg in range(0, 4):
        result = []
        for arg in args:
    #         print arg
            rlt = find_0_gaps_max_incr_analysis(hourly_vols[time_seg], hourly_maxs[time_seg], hourly_mins[time_seg], adj_look_back=arg[0], adj_vol_incr =arg[1], price_max_incr =arg[2], adj_next =arg[3])
            result.append(rlt)

        rpd = pd.DataFrame(result)
        rpd['id_ratio'] = rpd['increase']/rpd['decrease']
        rpd['total_gain'] = rpd['incr_total'] + rpd['decr_total']
        print "********* {} seg: {} *********".format(coin.upper(), time_seg)
        print rpd.sort_values('id_ratio', ascending=False).head(8)

In [7]:
base_obj = Base()

In [37]:
coins = ['DAG', 'PKG', 'MTC', 'SNTR', 'PCL', 'MET', 'BWX', 'CARD', 'UPP', 'ZCN', 'QNT', 'WEB', 'ESS', 'TOMO', 'TGAME']
for coin in coins:
    back_test_time_segment_volume_analysis(base_obj, coin, args)

1666 3332
********* DAG seg: 0 *********
   adj_look_back  adj_next  adj_vol_incr  decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  id_ratio  total_gain
0              2        10             6         0  -34.102515  0.002016         0           0       0.0              30    992       0.0  -34.102515
1              2        24             6         0  -47.733463  0.003024         0           0       0.0              30    992       0.0  -47.733463
2              2        10             6         0    0.000000  0.000504         0           0       0.0              50    992       0.0    0.000000
3              2        24             6         0    0.000000  0.000504         0           0       0.0              50    992       0.0    0.000000
4              2        10            12         0  -34.102515  0.002016         0           0       0.0              30    992       0.0  -34.102515
5              2        24            12         0  -47.733

********* PKG seg: 2 *********
   adj_look_back  adj_next  adj_vol_incr   decr_avg  decr_total  decrease   incr_avg  incr_total  increase  price_max_incr  steps  id_ratio  total_gain
3              2        24             6   0.000000    0.000000  0.004032  80.770749   80.770749  0.008065              50    124       2.0   80.770749
7              2        24            12   0.000000    0.000000  0.004032  80.770749   80.770749  0.008065              50    124       2.0   80.770749
1              2        24             6 -21.909669  -43.819338  0.016129  55.972727  111.945453  0.016129              30    124       1.0   68.126115
0              2        10             6 -22.844030  -45.688060  0.016129  41.667182   41.667182  0.008065              30    124       0.5   -4.020878
4              2        10            12 -22.844030  -45.688060  0.016129  41.667182   41.667182  0.008065              30    124       0.5   -4.020878
5              2        24            12 -21.909669  -43.

1666 3332
********* SNTR seg: 0 *********
   adj_look_back  adj_next  adj_vol_incr  decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  id_ratio  total_gain
0              2        10             6         0           0  0.004167         0           0       0.0              30    120       0.0           0
1              2        24             6         0           0  0.004167         0           0       0.0              30    120       0.0           0
2              2        10             6         0           0  0.004167         0           0       0.0              50    120       0.0           0
3              2        24             6         0           0  0.004167         0           0       0.0              50    120       0.0           0
4              2        10            12         0           0  0.004167         0           0       0.0              30    120       0.0           0
5              2        24            12         0        

********* PCL seg: 2 *********
   adj_look_back  adj_next  adj_vol_incr  decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  id_ratio  total_gain
0              2        10             6         0           0  0.001773         0           0       0.0              30    282       0.0           0
1              2        24             6         0           0  0.001773         0           0       0.0              30    282       0.0           0
2              2        10             6         0           0  0.001773         0           0       0.0              50    282       0.0           0
3              2        24             6         0           0  0.001773         0           0       0.0              50    282       0.0           0
4              2        10            12         0           0  0.001773         0           0       0.0              30    282       0.0           0
5              2        24            12         0           0  0.001

1287 2574
********* BWX seg: 0 *********
   adj_look_back  adj_next  adj_vol_incr  decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  id_ratio  total_gain
0              2        10             6         0           0  0.001348         0           0       0.0              30    371       0.0           0
1              2        24             6         0           0  0.001348         0           0       0.0              30    371       0.0           0
2              2        10             6         0           0  0.001348         0           0       0.0              50    371       0.0           0
3              2        24             6         0           0  0.001348         0           0       0.0              50    371       0.0           0
4              2        10            12         0           0  0.001348         0           0       0.0              30    371       0.0           0
5              2        24            12         0         

********* CARD seg: 2 *********
   adj_look_back  adj_next  adj_vol_incr  decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  id_ratio  total_gain
0              2        10             6         0           0  0.006329         0           0       0.0              30     79       0.0           0
1              2        24             6         0           0  0.006329         0           0       0.0              30     79       0.0           0
2              2        10             6         0           0  0.006329         0           0       0.0              50     79       0.0           0
3              2        24             6         0           0  0.006329         0           0       0.0              50     79       0.0           0
4              2        10            12         0           0  0.006329         0           0       0.0              30     79       0.0           0
5              2        24            12         0           0  0.00

1666 3332
********* ZCN seg: 0 *********
    adj_look_back  adj_next  adj_vol_incr   decr_avg  decr_total  decrease   incr_avg  incr_total  increase  price_max_incr  steps  id_ratio  total_gain
5               2        24            12   0.000000    0.000000  0.000662  46.412745  232.063724  0.006623              30    755      10.0  232.063724
3               2        24             6   0.000000    0.000000  0.000662  56.530529  169.591586  0.003974              50    755       6.0  169.591586
7               2        24            12   0.000000    0.000000  0.000662  56.530529  169.591586  0.003974              50    755       6.0  169.591586
1               2        24             6 -15.354706  -15.354706  0.001325  46.412745  232.063724  0.006623              30    755       5.0  216.709018
9               4        24             6   0.000000    0.000000  0.000662  44.065710   88.131421  0.002649              30    755       4.0   88.131421
13              4        24            12

********* QNT seg: 2 *********
    adj_look_back  adj_next  adj_vol_incr   decr_avg  decr_total  decrease   incr_avg  incr_total  increase  price_max_incr  steps  id_ratio  total_gain
1               2        24             6 -17.104955  -17.104955  0.004926  47.057971   94.115943  0.009852              30    203       2.0   77.010988
3               2        24             6   0.000000    0.000000  0.002463  61.078726   61.078726  0.004926              50    203       2.0   61.078726
5               2        24            12 -17.104955  -17.104955  0.004926  47.057971   94.115943  0.009852              30    203       2.0   77.010988
7               2        24            12   0.000000    0.000000  0.002463  61.078726   61.078726  0.004926              50    203       2.0   61.078726
9               4        24             6   0.000000    0.000000  0.002463  33.037217   33.037217  0.004926              30    203       2.0   33.037217
13              4        24            12   0.00000

587 1174
********* ESS seg: 0 *********
   adj_look_back  adj_next  adj_vol_incr  decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  id_ratio  total_gain
0              2        10             6         0           0  0.000956         0           0       0.0              30    523       0.0           0
1              2        24             6         0           0  0.000956         0           0       0.0              30    523       0.0           0
2              2        10             6         0           0  0.000956         0           0       0.0              50    523       0.0           0
3              2        24             6         0           0  0.000956         0           0       0.0              50    523       0.0           0
4              2        10            12         0           0  0.000956         0           0       0.0              30    523       0.0           0
5              2        24            12         0          

********* TOMO seg: 2 *********
   adj_look_back  adj_next  adj_vol_incr  decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  id_ratio  total_gain
0              2        10             6         0           0  0.001779         0           0       0.0              30    281       0.0           0
1              2        24             6         0           0  0.001779         0           0       0.0              30    281       0.0           0
2              2        10             6         0           0  0.001779         0           0       0.0              50    281       0.0           0
3              2        24             6         0           0  0.001779         0           0       0.0              50    281       0.0           0
4              2        10            12         0           0  0.001779         0           0       0.0              30    281       0.0           0
5              2        24            12         0           0  0.00

In [22]:
seg1 = int(1595/3)
seg2 = 2*seg1
    
print seg1, seg2

531 1062


In [57]:
adj_look_backs = [2, 4]
adj_vol_incrs = [ 6, 12]
price_max_incrs = [30, 50]
adj_nexts = [5, 7]

args_array = [adj_look_backs, adj_vol_incrs, price_max_incrs, adj_nexts]
args = list(itertools.product(*args_array))

In [39]:
for coin in coins:
    back_test_time_segment_volume_analysis(base_obj, coin, args)

1666 3332
********* DAG seg: 0 *********
   adj_look_back  adj_next  adj_vol_incr  decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  id_ratio  total_gain
0              2         5             6         0  -31.413785  0.002016         0           0       0.0              30    992       0.0  -31.413785
1              2         7             6         0  -31.413785  0.002016         0           0       0.0              30    992       0.0  -31.413785
2              2         5             6         0    0.000000  0.000504         0           0       0.0              50    992       0.0    0.000000
3              2         7             6         0    0.000000  0.000504         0           0       0.0              50    992       0.0    0.000000
4              2         5            12         0  -15.994624  0.001008         0           0       0.0              30    992       0.0  -15.994624
5              2         7            12         0  -15.994

********* PKG seg: 2 *********
   adj_look_back  adj_next  adj_vol_incr   decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  id_ratio  total_gain
1              2         7             6 -28.491749  -56.983498  0.016129   33.1995     33.1995  0.008065              30    124       0.5  -23.783998
5              2         7            12 -28.491749  -56.983498  0.016129   33.1995     33.1995  0.008065              30    124       0.5  -23.783998
0              2         5             6   0.000000  -71.104623  0.016129    0.0000      0.0000  0.000000              30    124       0.0  -71.104623
2              2         5             6   0.000000  -71.104623  0.016129    0.0000      0.0000  0.000000              50    124       0.0  -71.104623
3              2         7             6   0.000000  -56.983498  0.016129    0.0000      0.0000  0.000000              50    124       0.0  -56.983498
4              2         5            12   0.000000  -71.104623

1666 3332
********* SNTR seg: 0 *********
   adj_look_back  adj_next  adj_vol_incr  decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  id_ratio  total_gain
0              2         5             6         0           0  0.004167         0           0       0.0              30    120       0.0           0
1              2         7             6         0           0  0.004167         0           0       0.0              30    120       0.0           0
2              2         5             6         0           0  0.004167         0           0       0.0              50    120       0.0           0
3              2         7             6         0           0  0.004167         0           0       0.0              50    120       0.0           0
4              2         5            12         0           0  0.004167         0           0       0.0              30    120       0.0           0
5              2         7            12         0        

********* PCL seg: 2 *********
   adj_look_back  adj_next  adj_vol_incr  decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  id_ratio  total_gain
0              2         5             6         0           0  0.001773         0           0       0.0              30    282       0.0           0
1              2         7             6         0           0  0.001773         0           0       0.0              30    282       0.0           0
2              2         5             6         0           0  0.001773         0           0       0.0              50    282       0.0           0
3              2         7             6         0           0  0.001773         0           0       0.0              50    282       0.0           0
4              2         5            12         0           0  0.001773         0           0       0.0              30    282       0.0           0
5              2         7            12         0           0  0.001

1287 2574
********* BWX seg: 0 *********
   adj_look_back  adj_next  adj_vol_incr  decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  id_ratio  total_gain
0              2         5             6         0           0  0.001348         0           0       0.0              30    371       0.0           0
1              2         7             6         0           0  0.001348         0           0       0.0              30    371       0.0           0
2              2         5             6         0           0  0.001348         0           0       0.0              50    371       0.0           0
3              2         7             6         0           0  0.001348         0           0       0.0              50    371       0.0           0
4              2         5            12         0           0  0.001348         0           0       0.0              30    371       0.0           0
5              2         7            12         0         

********* CARD seg: 2 *********
   adj_look_back  adj_next  adj_vol_incr  decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  id_ratio  total_gain
0              2         5             6         0           0  0.006329         0           0       0.0              30     79       0.0           0
1              2         7             6         0           0  0.006329         0           0       0.0              30     79       0.0           0
2              2         5             6         0           0  0.006329         0           0       0.0              50     79       0.0           0
3              2         7             6         0           0  0.006329         0           0       0.0              50     79       0.0           0
4              2         5            12         0           0  0.006329         0           0       0.0              30     79       0.0           0
5              2         7            12         0           0  0.00

1666 3332
********* ZCN seg: 0 *********
    adj_look_back  adj_next  adj_vol_incr   decr_avg  decr_total  decrease   incr_avg  incr_total  increase  price_max_incr  steps  id_ratio  total_gain
9               4         7             6   0.000000    0.000000  0.000662  31.430449   31.430449  0.001325              30    755       2.0   31.430449
13              4         7            12   0.000000    0.000000  0.000662  31.430449   31.430449  0.001325              30    755       2.0   31.430449
5               2         7            12 -20.090693  -20.090693  0.001325  31.430449   31.430449  0.001325              30    755       1.0   11.339756
1               2         7             6 -17.722699  -35.445399  0.002649  31.430449   31.430449  0.001325              30    755       0.5   -4.014950
0               2         5             6   0.000000  -31.640179  0.002649   0.000000    0.000000  0.000000              30    755       0.0  -31.640179
2               2         5             6

********* QNT seg: 2 *********
    adj_look_back  adj_next  adj_vol_incr   decr_avg  decr_total  decrease   incr_avg  incr_total  increase  price_max_incr  steps  id_ratio  total_gain
9               4         7             6 -16.604296  -16.604296  0.004926  33.037217   33.037217  0.004926              30    203  1.000000   16.432921
12              4         5            12 -16.604296  -16.604296  0.004926  33.037217   33.037217  0.004926              30    203  1.000000   16.432921
13              4         7            12 -16.604296  -16.604296  0.004926  33.037217   33.037217  0.004926              30    203  1.000000   16.432921
8               4         5             6 -24.797739  -49.595478  0.009852  33.037217   33.037217  0.004926              30    203  0.500000  -16.558261
1               2         7             6 -19.233032  -57.699095  0.014778  33.037217   33.037217  0.004926              30    203  0.333333  -24.661878
4               2         5            12 -19.23303

588 1176
********* ESS seg: 0 *********
   adj_look_back  adj_next  adj_vol_incr  decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  id_ratio  total_gain
0              2         5             6         0           0  0.000963         0           0       0.0              30    519       0.0           0
1              2         7             6         0           0  0.000963         0           0       0.0              30    519       0.0           0
2              2         5             6         0           0  0.000963         0           0       0.0              50    519       0.0           0
3              2         7             6         0           0  0.000963         0           0       0.0              50    519       0.0           0
4              2         5            12         0           0  0.000963         0           0       0.0              30    519       0.0           0
5              2         7            12         0          

********* TOMO seg: 2 *********
   adj_look_back  adj_next  adj_vol_incr  decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  id_ratio  total_gain
0              2         5             6         0           0  0.001779         0           0       0.0              30    281       0.0           0
1              2         7             6         0           0  0.001779         0           0       0.0              30    281       0.0           0
2              2         5             6         0           0  0.001779         0           0       0.0              50    281       0.0           0
3              2         7             6         0           0  0.001779         0           0       0.0              50    281       0.0           0
4              2         5            12         0           0  0.001779         0           0       0.0              30    281       0.0           0
5              2         7            12         0           0  0.00

In [109]:
reload(Analytics)

<module 'Analytics' from 'Analytics.py'>

In [78]:
adj_look_backs = [2, 4, 6]
adj_vol_incrs = [ 3, 9]
price_max_incrs = [30, 50]
adj_nexts = [10, 24]

args_array = [adj_look_backs, adj_vol_incrs, price_max_incrs, adj_nexts]
args = list(itertools.product(*args_array))

In [79]:
coins = ['DAG', 'PKG', 'MTC', 'SNTR', 'PCL', 'MET', 'BWX', 'CARD', 'UPP', 'ZCN', 'QNT', 'WEB', 'ESS', 'TOMO', 'TGAME']

## try using next_min for decrease
for coin in coins:
    Analytics.back_test_time_segment_volume_analysis(base_obj, coin, args)

1666 3332
********* DAG seg: 0 *********
    adj_look_back  adj_next  adj_vol_incr  avg_incr_vol   decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  up_decr_ratio  id_ratio  total_gain
0               2        10             3             0 -16.822640 -185.049041  0.011011         0           0       0.0              30    999       0.010010       0.0 -185.049041
1               2        24             3             0 -17.985668 -395.684702  0.022022         0           0       0.0              30    999       0.016016       0.0 -395.684702
22              6        10             9             0   0.000000    0.000000  0.000501         0           0       0.0              50    999       0.000000       0.0    0.000000
21              6        24             9             0 -16.868704  -33.737408  0.002002         0           0       0.0              30    999       0.001001       0.0  -33.737408
20              6        10             9             

********* PKG seg: 1 *********
    adj_look_back  adj_next  adj_vol_incr  avg_incr_vol   decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  up_decr_ratio  id_ratio  total_gain
0               2        10             3             0 -40.252316  -80.504633  0.042553         0           0       0.0              30     47       0.042553       0.0  -80.504633
1               2        24             3             0 -40.252316  -80.504633  0.042553         0           0       0.0              30     47       0.042553       0.0  -80.504633
22              6        10             9             0   0.000000    0.000000  0.010638         0           0       0.0              50     47       0.000000       0.0    0.000000
21              6        24             9             0   0.000000    0.000000  0.010638         0           0       0.0              30     47       0.000000       0.0    0.000000
20              6        10             9             0   0.0000

********* MTC seg: 2 *********
    adj_look_back  adj_next  adj_vol_incr  avg_incr_vol   decr_avg  decr_total  decrease   incr_avg  incr_total  increase  price_max_incr  steps  up_decr_ratio  id_ratio  total_gain
4               2        10             9     34.034153 -15.084257  -15.084257  0.002315  37.500000   37.500000  0.002315              30    432       0.002315  1.000000   22.415743
5               2        24             9    107.778695 -19.507248  -58.521744  0.006944  44.493651   88.987303  0.004630              30    432       0.002315  0.666667   30.465558
1               2        24             3    124.011319 -18.166572 -163.499148  0.020833  46.768235  187.072941  0.009259              30    432       0.013889  0.444444   23.573794
0               2        10             3     34.034153 -17.678475  -88.392377  0.011574  37.500000   37.500000  0.002315              30    432       0.011574  0.200000  -50.892377
14              4        10             9      0.000000   0

********* SNTR seg: 3 *********
    adj_look_back  adj_next  adj_vol_incr  avg_incr_vol  decr_avg  decr_total  decrease   incr_avg  incr_total  increase  price_max_incr  steps  up_decr_ratio  id_ratio  total_gain
5               2        24             9    784.581914       0.0           0   0.00133  40.727275   40.727275   0.00266              30    376            0.0       2.0   40.727275
1               2        24             3    784.581914       0.0           0   0.00133  40.727275   40.727275   0.00266              30    376            0.0       2.0   40.727275
0               2        10             3      0.000000       0.0           0   0.00133   0.000000    0.000000   0.00000              30    376            0.0       0.0    0.000000
13              4        24             9      0.000000       0.0           0   0.00133   0.000000    0.000000   0.00000              30    376            0.0       0.0    0.000000
22              6        10             9      0.000000       0

1666 3332
********* MET seg: 0 *********
    adj_look_back  adj_next  adj_vol_incr  avg_incr_vol   decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  up_decr_ratio  id_ratio  total_gain
0               2        10             3             0   0.000000    0.000000  0.001185         0           0       0.0              30    422        0.00000       0.0    0.000000
1               2        24             3             0 -17.413008  -17.413008  0.002370         0           0       0.0              30    422        0.00237       0.0  -17.413008
22              6        10             9             0   0.000000    0.000000  0.001185         0           0       0.0              50    422        0.00000       0.0    0.000000
21              6        24             9             0   0.000000    0.000000  0.001185         0           0       0.0              30    422        0.00000       0.0    0.000000
20              6        10             9             

...

********* CARD seg: 2 *********
    adj_look_back  adj_next  adj_vol_incr  avg_incr_vol   decr_avg  decr_total  decrease      incr_avg    incr_total  increase  price_max_incr  steps  up_decr_ratio  id_ratio    total_gain
3               2        24             3    719.429871 -30.528172  -61.056345  0.025316  8.352997e+24  1.670599e+25  0.025316              50     79       0.012658       1.0  1.670599e+25
1               2        24             3    719.429871 -30.528172  -61.056345  0.025316  8.352997e+24  1.670599e+25  0.025316              30     79       0.012658       1.0  1.670599e+25
0               2        10             3    444.852570 -27.686446  -55.372891  0.025316  6.978367e+24  6.978367e+24  0.012658              30     79       0.012658       0.5  6.978367e+24
2               2        10             3    444.852570 -27.686446  -55.372891  0.025316  6.978367e+24  6.978367e+24  0.012658              50     79       0.012658       0.5  6.978367e+24
14              4      

********* UPP seg: 3 *********
    adj_look_back  adj_next  adj_vol_incr  avg_incr_vol   decr_avg  decr_total  decrease   incr_avg  incr_total  increase  price_max_incr  steps  up_decr_ratio  id_ratio  total_gain
3               2        24             3     95.065934   0.000000    0.000000  0.000353  51.023638   51.023638  0.000705              50   1418       0.000000  2.000000   51.023638
1               2        24             3    137.431275 -18.700385 -411.408460  0.015515  40.627314  121.881941  0.002116              30   1418       0.014810  0.136364 -289.526519
9               4        24             3     96.817685 -18.022892 -288.366275  0.011283  40.091743   40.091743  0.000705              30   1418       0.010578  0.062500 -248.274532
5               2        24             9    220.410205 -17.919024 -304.623405  0.011989  30.766560   30.766560  0.000705              30   1418       0.011283  0.058824 -273.856845
0               2        10             3      0.000000 -17

864 1728
********* QNT seg: 0 *********
    adj_look_back  adj_next  adj_vol_incr  avg_incr_vol   decr_avg  decr_total  decrease   incr_avg  incr_total  increase  price_max_incr  steps  up_decr_ratio  id_ratio  total_gain
17              6        24             3     17.330510 -37.477432  -74.954865  0.004651  46.217936   46.217936  0.002326              30    430       0.000000  0.500000  -28.736928
13              4        24             9     20.645691 -33.823190 -169.115951  0.011628  44.563514   89.127027  0.004651              30    430       0.002326  0.400000  -79.988924
5               2        24             9     23.571125 -37.192538 -446.310458  0.027907  42.431093  169.724371  0.009302              30    430       0.013953  0.333333 -276.586087
9               4        24             3     19.649016 -33.665241 -302.987169  0.020930  45.495764  136.487291  0.006977              30    430       0.006977  0.333333 -166.499878
1               2        24             3     23.5

********* WEB seg: 1 *********
    adj_look_back  adj_next  adj_vol_incr  avg_incr_vol   decr_avg   decr_total  decrease   incr_avg   incr_total  increase  price_max_incr  steps  up_decr_ratio  id_ratio  total_gain
16              6        10             3      4.721956 -37.361699  -523.063790  0.009950  72.502863  1232.548663  0.012082              30   1407       0.006397  1.214286  709.484873
0               2        10             3      4.055621 -34.561410 -1071.403697  0.022033  63.912504  1981.287633  0.022033              30   1407       0.013504  1.000000  909.883936
20              6        10             9      4.496340 -37.764626  -490.940134  0.009240  79.819431  1037.652606  0.009240              30   1407       0.006397  1.000000  546.712472
4               2        10             9      3.830422 -34.814890 -1009.631800  0.020611  65.014955  1885.433701  0.020611              30   1407       0.013504  1.000000  875.801901
8               4        10             3      4.

SSLError: HTTPSConnectionPool(host='api.idex.market', port=443): Max retries exceeded with url: /returnTradeHistory (Caused by SSLError(SSLError("bad handshake: SysCallError(54, 'ECONNRESET')",),))

In [81]:
for coin in coins:
    Analytics.back_test_time_segment_volume_analysis(base_obj, coin, args)

1666 3332
********* DAG seg: 0 *********
   adj_look_back  adj_next  adj_vol_incr  avg_incr_vol  decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  up_decr_ratio  id_ratio  total_gain
0              2        10             3             0         0           0       0.0         0           0       0.0              30    999            0.0       NaN           0
1              2        24             3             0         0           0       0.0         0           0       0.0              30    999            0.0       NaN           0
2              2        10             3             0         0           0       0.0         0           0       0.0              50    999            0.0       NaN           0
3              2        24             3             0         0           0       0.0         0           0       0.0              50    999            0.0       NaN           0
4              2        10             9             0         0

********* PKG seg: 1 *********
    adj_look_back  adj_next  adj_vol_incr  avg_incr_vol   decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  up_decr_ratio  id_ratio  total_gain
0               2        10             3             0 -25.494277  -25.494277  0.021277         0           0       0.0              30     47       0.021277       0.0  -25.494277
2               2        10             3             0 -25.494277  -25.494277  0.021277         0           0       0.0              50     47       0.021277       0.0  -25.494277
8               4        10             3             0 -25.494277  -25.494277  0.021277         0           0       0.0              30     47       0.021277       0.0  -25.494277
10              4        10             3             0 -25.494277  -25.494277  0.021277         0           0       0.0              50     47       0.021277       0.0  -25.494277
1               2        24             3             0   0.0000

********* MTC seg: 2 *********
   adj_look_back  adj_next  adj_vol_incr  avg_incr_vol  decr_avg  decr_total  decrease   incr_avg  incr_total  increase  price_max_incr  steps  up_decr_ratio  id_ratio  total_gain
0              2        10             3     34.034153         0           0       0.0  37.500000   37.500000  0.002315              30    432            0.0       inf   37.500000
1              2        24             3    124.011319         0           0       0.0  46.768235  187.072941  0.009259              30    432            0.0       inf  187.072941
4              2        10             9     34.034153         0           0       0.0  37.500000   37.500000  0.002315              30    432            0.0       inf   37.500000
5              2        24             9    107.778695         0           0       0.0  44.493651   88.987303  0.004630              30    432            0.0       inf   88.987303
2              2        10             3      0.000000         0     

********* SNTR seg: 3 *********
   adj_look_back  adj_next  adj_vol_incr  avg_incr_vol  decr_avg  decr_total  decrease   incr_avg  incr_total  increase  price_max_incr  steps  up_decr_ratio  id_ratio  total_gain
1              2        24             3    784.581914         0           0       0.0  40.727275   40.727275   0.00266              30    376            0.0       inf   40.727275
5              2        24             9    784.581914         0           0       0.0  40.727275   40.727275   0.00266              30    376            0.0       inf   40.727275
0              2        10             3      0.000000         0           0       0.0   0.000000    0.000000   0.00000              30    376            0.0       NaN    0.000000
2              2        10             3      0.000000         0           0       0.0   0.000000    0.000000   0.00000              50    376            0.0       NaN    0.000000
3              2        24             3      0.000000         0    

1666 3332
********* MET seg: 0 *********
   adj_look_back  adj_next  adj_vol_incr  avg_incr_vol  decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  up_decr_ratio  id_ratio  total_gain
0              2        10             3             0         0           0       0.0         0           0       0.0              30    422            0.0       NaN           0
1              2        24             3             0         0           0       0.0         0           0       0.0              30    422            0.0       NaN           0
2              2        10             3             0         0           0       0.0         0           0       0.0              50    422            0.0       NaN           0
3              2        24             3             0         0           0       0.0         0           0       0.0              50    422            0.0       NaN           0
4              2        10             9             0         0

...

********* UPP seg: 3 *********
   adj_look_back  adj_next  adj_vol_incr  avg_incr_vol  decr_avg  decr_total  decrease   incr_avg  incr_total  increase  price_max_incr  steps  up_decr_ratio  id_ratio  total_gain
1              2        24             3    137.431275         0           0       0.0  40.627314  121.881941  0.002116              30   1418            0.0       inf  121.881941
3              2        24             3     95.065934         0           0       0.0  51.023638   51.023638  0.000705              50   1418            0.0       inf   51.023638
5              2        24             9    220.410205         0           0       0.0  30.766560   30.766560  0.000705              30   1418            0.0       inf   30.766560
9              4        24             3     96.817685         0           0       0.0  40.091743   40.091743  0.000705              30   1418            0.0       inf   40.091743
0              2        10             3      0.000000         0     

864 1728
********* QNT seg: 0 *********
    adj_look_back  adj_next  adj_vol_incr  avg_incr_vol   decr_avg  decr_total  decrease   incr_avg  incr_total  increase  price_max_incr  steps  up_decr_ratio  id_ratio  total_gain
1               2        24             3     23.571125   0.000000    0.000000  0.000000  42.431093  169.724371  0.009302              30    430       0.000000       inf  169.724371
5               2        24             9     23.571125   0.000000    0.000000  0.000000  42.431093  169.724371  0.009302              30    430       0.000000       inf  169.724371
9               4        24             3     19.649016   0.000000    0.000000  0.000000  45.495764  136.487291  0.006977              30    430       0.000000       inf  136.487291
13              4        24             9     20.645691   0.000000    0.000000  0.000000  44.563514   89.127027  0.004651              30    430       0.000000       inf   89.127027
17              6        24             3     17.3

********* WEB seg: 1 *********
    adj_look_back  adj_next  adj_vol_incr  avg_incr_vol  decr_avg  decr_total  decrease    incr_avg   incr_total  increase  price_max_incr  steps  up_decr_ratio  id_ratio   total_gain
23              6        24             9      8.568384       0.0         0.0       0.0  130.591107   914.137751  0.004975              50   1407            0.0       inf   914.137751
22              6        10             9     12.210063       0.0         0.0       0.0  176.953053   707.812211  0.002843              50   1407            0.0       inf   707.812211
19              6        24             3      8.844038       0.0         0.0       0.0  121.031787  1089.286081  0.006397              50   1407            0.0       inf  1089.286081
18              6        10             3     10.328937       0.0         0.0       0.0  135.528146   813.168874  0.004264              50   1407            0.0       inf   813.168874
15              4        24             9      9.

********* ESS seg: 2 *********
   adj_look_back  adj_next  adj_vol_incr  avg_incr_vol  decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  up_decr_ratio  id_ratio  total_gain
0              2        10             3             0         0           0       0.0         0           0       0.0              30    131            0.0       NaN           0
1              2        24             3             0         0           0       0.0         0           0       0.0              30    131            0.0       NaN           0
2              2        10             3             0         0           0       0.0         0           0       0.0              50    131            0.0       NaN           0
3              2        24             3             0         0           0       0.0         0           0       0.0              50    131            0.0       NaN           0
4              2        10             9             0         0          

********* TOMO seg: 3 *********
   adj_look_back  adj_next  adj_vol_incr  avg_incr_vol  decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  up_decr_ratio  id_ratio  total_gain
0              2        10             3             0         0           0       0.0         0           0       0.0              30   1426            0.0       NaN           0
1              2        24             3             0         0           0       0.0         0           0       0.0              30   1426            0.0       NaN           0
2              2        10             3             0         0           0       0.0         0           0       0.0              50   1426            0.0       NaN           0
3              2        24             3             0         0           0       0.0         0           0       0.0              50   1426            0.0       NaN           0
4              2        10             9             0         0         

In [106]:
base_obj = Base()

In [107]:
adj_look_backs = [2, 4, 6]
adj_vol_incrs = [ 3, 9]
price_max_incrs = [30, 50]
adj_nexts = [10, 24]

args_array = [adj_look_backs, adj_vol_incrs, price_max_incrs, adj_nexts]
args = list(itertools.product(*args_array))

In [111]:
coins = ['REM']

## try using next_min for decrease
for coin in coins:
    Analytics.back_test_time_segment_volume_analysis(base_obj, coin, args)

1666 3332
********* REM seg: 0 *********
   adj_look_back  adj_next  adj_vol_incr  avg_incr_vol  decr_avg  decr_total  decrease  incr_avg  incr_total  increase  price_max_incr  steps  up_decr_ratio  id_ratio  total_gain
0              2        10             3             0         0           0       0.0         0           0       0.0              30    107            0.0       NaN           0
1              2        24             3             0         0           0       0.0         0           0       0.0              30    107            0.0       NaN           0
2              2        10             3             0         0           0       0.0         0           0       0.0              50    107            0.0       NaN           0
3              2        24             3             0         0           0       0.0         0           0       0.0              50    107            0.0       NaN           0
4              2        10             9             0         0

## Cascading groupBy

In [14]:
dag_hours = dag_hist.groupby(['bid_date', 'bid_hour'])

In [19]:
dag_hours.sum().sort_index(level=['bid_date', 'bid_hour'], ascending=False)

price     timestamp       total  bid_minute
bid_date   bid_hour                                                
2018-07-04 17        0.000055    1530698899    0.724284           8
           16        0.000055    1530697626    0.157553          47
           15        0.000052    1530691646    0.130555           7
           14        0.000218    6122760343    5.413076         163
           13        0.000156    4592060072    0.361920         133
           12        0.000202    6122729038   10.998779         122
           9         0.000778   22960094851   30.870964         836
           8         0.000528   15306676214    3.579644         268
           6         0.000055    1530661735    0.652484          48
           5         0.000161    4591974510    1.147516         147
           4         0.000314    9183929096   11.079442         324
           3         0.000717   19898459845   10.252982         590
           0         0.001030   29082144591   18.105512         625
2018-07-03 23        0.001641   44388417287   44.983633         706
           22        0.000854   21428847131   21.179341         446
           21        0.001305   32143202581   17.695400         793
           20        0.001248   29081891561   19.656120         960
           19        0.000559   13775599122   10.962403         435
           18        0.002337   56632830634   50.590107         878
           17        0.005806  131632835598  143.122706        3351
           16        0.001785   44387709693   77.875654        1088
           15        0.003085   76530308635  123.583383        1119
           14        0.000867   22959050963   23.898947         542
           13        0.000226    6122396547    5.865491         101
           12        0.000057    1530594591    0.074996           9
...                       ...           ...         ...         ...
2018-06-20 20        0.001965   48944046811   48.570975         980
           19        0.003079   81063356126  101.614146        1104
           18        0.001682   44355323545   47.340139         723
           17        0.004347  114711798042  308.724060        2261
           16        0.003285   78003832943  148.658191        1440
           15        0.002004   44355008263   34.984881         681
           14        0.004716   97886663224  337.879943        1155
           13        0.008006  160595067319  379.528009        4196
           12        0.004745  110122034029  222.899835        2231
           11        0.001399   36707266254   26.704222         877
           10        0.001147   30589327527   14.282554         914
           9         0.002356   62707946463   58.862448        1419
           8         0.002247   62707783368   88.986759        1160
           7         0.001627   44354166918   18.696246         581
           6         0.004160  110120441255  300.645783        1619
           5         0.005478  126944084430  328.756407        1614
           4         0.006836  148356034635  154.533362        2626
           3         0.006067  143767397392  169.678779        3088
           2         0.006527  157531940166  178.216710        2553
           1         0.006592  169767010393  158.707292        2389
           0         0.003776  116236625253  191.596370        2095
2018-06-19 23        0.007104  212590174582  240.504803        3906
           22        0.003028  100941892812   73.443488        2401
           21        0.001656   58117884395   58.869105         750
           20        0.001073   38235394772   52.071076        1067

[320 rows x 4 columns]

In [19]:
dag_hours = dag_agg.sum().groupby(['bid_date', 'bid_hour'])

In [23]:
dag_hours.sum().sort_index(level=['bid_date', 'bid_hour'], ascending=False)

price     timestamp       total
bid_date   bid_hour                                    
2018-06-28 5         0.000306    7650820108    2.654111
           4         0.000188    4590477839    0.652275
           3         0.000246    6120622073    0.583402
           2         0.001121   27542754570   40.260292
           1         0.000061    1530148995    0.077573
2018-06-27 23        0.000061    1530140905    0.533803
           22        0.000308    7650700523    8.378811
           21        0.000246    6120539725    2.635728
           20        0.000236    6120528429    1.254820
           18        0.000355    9180740908    2.923567
           16        0.000880   22951752941   45.495062
           15        0.000169    4590343161   20.680554
           14        0.000397   10710765416    5.967751
           12        0.000348    9180619517    1.814509
           11        0.000400   10710689678   22.239928
           10        0.000234    6120384369    1.034568
           9         0.000955   26011546678   37.040131
           8         0.000471   12240695752    9.194648
           7         0.000120    3060167924    3.270900
           6         0.000820   19891051668   14.476536
           4         0.000620   15300738729   15.054005
           3         0.000060    1530070973   10.000000
           2         0.000060    1530066221    0.110842
           1         0.000299    7650310911   12.683110
           0         0.000475   12240471130    8.539543
...                       ...           ...         ...
2018-06-20 1         0.002247   62707783368   88.986759
           0         0.001627   44354166918   18.696246
2018-06-19 23        0.004160  110120441255  300.645783
           22        0.005478  126944084430  328.756407
           21        0.006836  148356034635  154.533362
           20        0.006067  143767397392  169.678779
           19        0.006527  157531940166  178.216710
           18        0.006592  169767010393  158.707292
           17        0.003776  116236625253  191.596370
           16        0.007104  212590174582  240.504803
           15        0.003028  100941892812   73.443488
           14        0.001656   58117884395   58.869105
           13        0.003336  116235509248  165.937513
           12        0.003600  128470555485  264.026069
           11        0.002908   99411541209  130.931690
           10        0.003857  128469959153  196.183023
           9         0.002608   64234820621   86.233519
           8         0.002050   41293719023   34.783522
           7         0.001857   35176057479   64.957736
           6         0.002375   41293524574   45.203080
           5         0.002359   39764060498   34.507023
           4         0.001498   22940733820   27.757051
           3         0.002371   36705094919   22.264917
           2         0.001492   22940618316   35.420183
           1         0.002660   39763668540   30.664665

[210 rows x 3 columns]

In [22]:
df01 = dag_hours.sum().loc["2018-07-04", 9]

In [23]:
df01

price         7.781501e-04
timestamp     2.296009e+10
total         3.087096e+01
bid_minute    8.360000e+02
Name: (2018-07-04, 9), dtype: float64

In [24]:
df01['price']

0.00077815009894156894

In [86]:
for key, value in enumerate(df01):
    print "{}: {}".format(key, value)

0: 0.000246442838526
1: 6120622073.0
2: 0.583402384802
3: 46.0


In [ ]:
dag_hours['time_stamp']

## Aggregate - mix, max, std - filter by index

In [42]:
dag_hours_details = dag_hours.agg({'price': ['min', 'max', 'mean', 'std'],
                                  'total': ['sum', 'min', 'max', 'mean', 'std'],
                                   'timestamp': ['count', 'min', 'max']
                                 })

NameError: name 'dag_hours' is not defined

In [16]:
dag_hours_details

timestamp                             price                                        total                                         
                        count         min         max       min       max      mean           std        sum       min        max      mean       std
bid_date   bid_hour                                                                                                                                  
2018-06-08 13               2  1528440705  1528440705  0.000002  0.000002  0.000002  0.000000e+00   0.346220  0.146220   0.200000  0.173110  0.038028
           15               1  1528446357  1528446357  0.000002  0.000002  0.000002           NaN   0.287250  0.287250   0.287250  0.287250       NaN
           20               4  1528464458  1528466084  0.000002  0.000002  0.000002  5.000000e-09   2.351815  0.309400   1.164800  0.587954  0.403454
           21              10  1528467826  1528469540  0.000002  0.000002  0.000002  2.436961e-07   1.653282  0.065792   0.332500  0.165328  0.084238
           22               9  1528470014  1528473459  0.000002  0.000002  0.000002  4.795832e-10   1.507221  0.051300   0.592759  0.167469  0.165650
           23               1  1528474955  1528474955  0.000002  0.000002  0.000002           NaN   0.151000  0.151000   0.151000  0.151000       NaN
2018-06-09 10               8  1528513302  1528515735  0.000002  0.000003  0.000002  9.800753e-08   2.716291  0.150100   0.582805  0.339536  0.151593
           13               2  1528526354  1528526892  0.000002  0.000002  0.000002  8.838835e-08   0.710626  0.159250   0.551376  0.355313  0.277275
           15               1  1528534792  1528534792  0.000002  0.000002  0.000002           NaN   0.232145  0.232145   0.232145  0.232145       NaN
           16               4  1528535066  1528536257  0.000002  0.000002  0.000002  2.982588e-08   0.884572  0.127592   0.281430  0.221143  0.072557
           18               1  1528543105  1528543105  0.000002  0.000002  0.000002           NaN   0.320962  0.320962   0.320962  0.320962       NaN
           19               2  1528549125  1528549136  0.000002  0.000002  0.000002  1.414214e-09   0.731415  0.274665   0.456750  0.365707  0.128754
           20               6  1528549388  1528551854  0.000002  0.000002  0.000002  2.658320e-08   1.458070  0.157000   0.465009  0.243012  0.115122
           21               1  1528553770  1528553770  0.000002  0.000002  0.000002           NaN   0.251941  0.251941   0.251941  0.251941       NaN
2018-06-10 3                1  1528576494  1528576494  0.000002  0.000002  0.000002           NaN   0.203283  0.203283   0.203283  0.203283       NaN
           5                1  1528585128  1528585128  0.000002  0.000002  0.000002           NaN   0.127680  0.127680   0.127680  0.127680       NaN
           12               3  1528610126  1528610371  0.000002  0.000002  0.000002  0.000000e+00   1.247750  0.390600   0.461900  0.415917  0.039891
           13               2  1528612247  1528612266  0.000002  0.000002  0.000002  1.414214e-08   0.477645  0.200608   0.277037  0.238823  0.054043
           18               4  1528630806  1528631727  0.000002  0.000002  0.000002  9.469248e-08   1.254263  0.155000   0.564000  0.313566  0.193161
           19               1  1528634535  1528634535  0.000002  0.000002  0.000002           NaN   0.600000  0.600000   0.600000  0.600000       NaN
           20               3  1528637627  1528638011  0.000002  0.000002  0.000002  3.122499e-07   0.520750  0.120750   0.243680  0.173583  0.063257
           21               1  1528639630  1528639630  0.000002  0.000002  0.000002           NaN   0.296732  0.296732   0.296732  0.296732       NaN
           22               2  1528644832  1528646014  0.000002  0.000002  0.000002  2.828427e-08   0.369161  0.164000   0.205161  0.184580  0.029105
2018-06-11 3                1  1528662194  1528662194  0.000002  0.000002  0.000002           NaN   0.283100  0.283100  

### Get open close prices

In [29]:
dag_hours_details['price_stick'] = dag_hours_details[('price', 'max')] - dag_hours_details[('price', 'min')]

In [30]:
dag_hours_details

timestamp                             price                                        total                                            price_stick
                        count         min         max       min       max      mean           std        sum       min        max      mean       std              
bid_date   bid_hour                                                                                                                                                
2018-06-08 13               2  1528440705  1528440705  0.000002  0.000002  0.000002  0.000000e+00   0.346220  0.146220   0.200000  0.173110  0.038028  0.000000e+00
           15               1  1528446357  1528446357  0.000002  0.000002  0.000002           NaN   0.287250  0.287250   0.287250  0.287250       NaN  0.000000e+00
           20               4  1528464458  1528466084  0.000002  0.000002  0.000002  5.000000e-09   2.351815  0.309400   1.164800  0.587954  0.403454  1.000000e-08
           21              10  1528467826  1528469540  0.000002  0.000002  0.000002  2.436961e-07   1.653282  0.065792   0.332500  0.165328  0.084238  5.900000e-07
           22               9  1528470014  1528473459  0.000002  0.000002  0.000002  4.795832e-10   1.507221  0.051300   0.592759  0.167469  0.165650  1.000000e-09
           23               1  1528474955  1528474955  0.000002  0.000002  0.000002           NaN   0.151000  0.151000   0.151000  0.151000       NaN  0.000000e+00
2018-06-09 10               8  1528513302  1528515735  0.000002  0.000003  0.000002  9.800753e-08   2.716291  0.150100   0.582805  0.339536  0.151593  2.200000e-07
           13               2  1528526354  1528526892  0.000002  0.000002  0.000002  8.838835e-08   0.710626  0.159250   0.551376  0.355313  0.277275  1.250000e-07
           15               1  1528534792  1528534792  0.000002  0.000002  0.000002           NaN   0.232145  0.232145   0.232145  0.232145       NaN  0.000000e+00
           16               4  1528535066  1528536257  0.000002  0.000002  0.000002  2.982588e-08   0.884572  0.127592   0.281430  0.221143  0.072557  6.500000e-08
           18               1  1528543105  1528543105  0.000002  0.000002  0.000002           NaN   0.320962  0.320962   0.320962  0.320962       NaN  0.000000e+00
           19               2  1528549125  1528549136  0.000002  0.000002  0.000002  1.414214e-09   0.731415  0.274665   0.456750  0.365707  0.128754  2.000000e-09
           20               6  1528549388  1528551854  0.000002  0.000002  0.000002  2.658320e-08   1.458070  0.157000   0.465009  0.243012  0.115122  7.000000e-08
           21               1  1528553770  1528553770  0.000002  0.000002  0.000002           NaN   0.251941  0.251941   0.251941  0.251941       NaN  0.000000e+00
2018-06-10 3                1  1528576494  1528576494  0.000002  0.000002  0.000002           NaN   0.203283  0.203283   0.203283  0.203283       NaN  0.000000e+00
           5                1  1528585128  1528585128  0.000002  0.000002  0.000002           NaN   0.127680  0.127680   0.127680  0.127680       NaN  0.000000e+00
           12               3  1528610126  1528610371  0.000002  0.000002  0.000002  0.000000e+00   1.247750  0.390600   0.461900  0.415917  0.039891  0.000000e+00
           13               2  1528612247  1528612266  0.000002  0.000002  0.000002  1.414214e-08   0.477645  0.200608   0.277037  0.238823  0.054043  2.000000e-08
           18               4  1528630806  1528631727  0.000002  0.000002  0.000002  9.469248e-08   1.254263  0.155000   0.564000  0.313566  0.193161  1.900000e-07
           19               1  1528634535  1528634535  0.000002  0.000002  0.000002           NaN   0.600000  0.600000   0.600000  0.600000       NaN  0.000000e+00
           20               3  1528637627  1528638011  0.000002  0.000002  0.000002  3.122499e-07   0.520750  0.120750   0.243680  0.173583  0.063257  6.000000e-07
           21               1  1528639630  1528639630  0.000002  0.000002  0.000002 

In [31]:
hist_join[['min_timestamp', 'open_price']] = dag_hist[['timestamp','price']]
dag_hours_details['min_ts'] = dag_hours_details[('timestamp', 'min')]
pd.merge(dag_hours_details, hist_join, left_on='min_ts', right_on='min_timestamp', how='left')

,"(timestamp, count)","(timestamp, min)","(timestamp, max)","(price, min)","(price, max)","(price, mean)","(price, std)","(total, sum)","(total, min)","(total, max)","(total, mean)","(total, std)","(price_stick, )","(min_ts, )",timestamp,price,min_timestamp,open_price
0,2,1528440705,1528440705,0.000002,0.000002,0.000002,0.000000e+00,0.346220,0.146220,0.200000,0.173110,0.038028,0.000000e+00,1528440705,1528440705,0.000002,1528440705,0.000002
1,2,1528440705,1528440705,0.000002,0.000002,0.000002,0.000000e+00,0.346220,0.146220,0.200000,0.173110,0.038028,0.000000e+00,1528440705,1528440705,0.000002,1528440705,0.000002
2,1,1528446357,1528446357,0.000002,0.000002,0.000002,NaN,0.287250,0.287250,0.287250,0.287250,NaN,0.000000e+00,1528446357,1528446357,0.000002,1528446357,0.000002
3,4,1528464458,1528466084,0.000002,0.000002,0.000002,5.000000e-09,2.351815,0.309400,1.164800,0.587954,0.403454,1.000000e-08,1528464458,1528464458,0.000002,1528464458,0.000002
4,10,1528467826,1528469540,0.000002,0.000002,0.000002,2.436961e-07,1.653282,0.065792,0.332500,0.165328,0.084238,5.900000e-07,1528467826,1528467826,0.000002,1528467826,0.000002
5,9,1528470014,1528473459,0.000002,0.000002,0.000002,4.795832e-10,1.507221,0.051300,0.592759,0.167469,0.165650,1.000000e-09,1528470014,1528470014,0.000002,1528470014,0.000002
6,1,1528474955,1528474955,0.000002,0.000002,0.000002,NaN,0.151000,0.151000,0.151000,0.151000,NaN,0.000000e+00,1528474955,1528474955,0.000002,1528474955,0.000002
7,8,1528513302,1528515735,0.000002,0.000003,0.000002,9.800753e-08,2.716291,0.150100,0.582805,0.339536,0.151593,2.200000e-07,1528513302,1528513302,0.000002,1528513302,0.000002
8,2,1528526354,1528526892,0.000002,0.000002,0.000002,8.838835e-08,0.710626,0.159250,0.551376,0.355313,0.277275,1.250000e-07,1528526354,1528526354,0.000002,1528526354,0.000002
9,1,1528534792,1528534792,0.000002,0.000002,0.000002,NaN,0.232145,0.232145,0.232145,0.232145,NaN,0.000000e+00,1528534792,1528534792,0.000002,1528534792,0.000002


In [32]:
hist_join1[['max_timestamp', 'close_price']] = dag_hist[['timestamp','price']]
dag_hours_details['max_ts'] = dag_hours_details[('timestamp', 'max')]
pd.merge(dag_hours_details, hist_join1, left_on='max_ts', right_on='max_timestamp', how='left')

NameError: name 'hist_join1' is not defined

In [33]:
dag_hours_details

timestamp                             price                                        total                                            price_stick      min_ts
                        count         min         max       min       max      mean           std        sum       min        max      mean       std                          
bid_date   bid_hour                                                                                                                                                            
2018-06-08 13               2  1528440705  1528440705  0.000002  0.000002  0.000002  0.000000e+00   0.346220  0.146220   0.200000  0.173110  0.038028  0.000000e+00  1528440705
           15               1  1528446357  1528446357  0.000002  0.000002  0.000002           NaN   0.287250  0.287250   0.287250  0.287250       NaN  0.000000e+00  1528446357
           20               4  1528464458  1528466084  0.000002  0.000002  0.000002  5.000000e-09   2.351815  0.309400   1.164800  0.587954  0.403454  1.000000e-08  1528464458
           21              10  1528467826  1528469540  0.000002  0.000002  0.000002  2.436961e-07   1.653282  0.065792   0.332500  0.165328  0.084238  5.900000e-07  1528467826
           22               9  1528470014  1528473459  0.000002  0.000002  0.000002  4.795832e-10   1.507221  0.051300   0.592759  0.167469  0.165650  1.000000e-09  1528470014
           23               1  1528474955  1528474955  0.000002  0.000002  0.000002           NaN   0.151000  0.151000   0.151000  0.151000       NaN  0.000000e+00  1528474955
2018-06-09 10               8  1528513302  1528515735  0.000002  0.000003  0.000002  9.800753e-08   2.716291  0.150100   0.582805  0.339536  0.151593  2.200000e-07  1528513302
           13               2  1528526354  1528526892  0.000002  0.000002  0.000002  8.838835e-08   0.710626  0.159250   0.551376  0.355313  0.277275  1.250000e-07  1528526354
           15               1  1528534792  1528534792  0.000002  0.000002  0.000002           NaN   0.232145  0.232145   0.232145  0.232145       NaN  0.000000e+00  1528534792
           16               4  1528535066  1528536257  0.000002  0.000002  0.000002  2.982588e-08   0.884572  0.127592   0.281430  0.221143  0.072557  6.500000e-08  1528535066
           18               1  1528543105  1528543105  0.000002  0.000002  0.000002           NaN   0.320962  0.320962   0.320962  0.320962       NaN  0.000000e+00  1528543105
           19               2  1528549125  1528549136  0.000002  0.000002  0.000002  1.414214e-09   0.731415  0.274665   0.456750  0.365707  0.128754  2.000000e-09  1528549125
           20               6  1528549388  1528551854  0.000002  0.000002  0.000002  2.658320e-08   1.458070  0.157000   0.465009  0.243012  0.115122  7.000000e-08  1528549388
           21               1  1528553770  1528553770  0.000002  0.000002  0.000002           NaN   0.251941  0.251941   0.251941  0.251941       NaN  0.000000e+00  1528553770
2018-06-10 3                1  1528576494  1528576494  0.000002  0.000002  0.000002           NaN   0.203283  0.203283   0.203283  0.203283       NaN  0.000000e+00  1528576494
           5                1  1528585128  1528585128  0.000002  0.000002  0.000002           NaN   0.127680  0.127680   0.127680  0.127680       NaN  0.000000e+00  1528585128
           12               3  1528610126  1528610371  0.000002  0.000002  0.000002  0.000000e+00   1.247750  0.390600   0.461900  0.415917  0.039891  0.000000e+00  1528610126
           13               2  1528612247  1528612266  0.000002  0.000002  0.000002  1.414214e-08   0.477645  0.200608   0.277037  0.238823  0.054043  2.000000e-08  1528612247
           18               4  1528630806  1528631727  0.000002  0.000002  0.000002  9.469248e-08   1.254263  0.155000   0.564000  0.313566  0.193161  1.900000e-07  1528630806
           19               1  1528634535  1528634535  0.000002  0.000002  0.000002           NaN   0.600000  0.600000   0.600000  0.600000       Na

In [39]:
dag_hours_details[('price', 'open')] = dag_hist[dag_hist['timestamp'] == dag_hours_details[('timestamp', 'min')]]['price']

ValueError: Can only compare identically-labeled Series objects

In [40]:
dag_hist['timestamp']

0      1531927892
1      1531927366
2      1531927366
3      1531927366
4      1531927366
5      1531926666
6      1531926591
7      1531926499
8      1531925194
9      1531925194
10     1531924596
11     1531923971
12     1531923857
13     1531923850
14     1531923748
15     1531923467
16     1531923449
17     1531923233
18     1531922504
19     1531922504
20     1531922504
21     1531922504
22     1531922504
23     1531922504
24     1531922504
          ...    
854    1528472557
855    1528472361
856    1528472268
857    1528471963
858    1528471479
859    1528471281
860    1528471277
861    1528470014
862    1528469540
863    1528469071
864    1528468892
865    1528468790
866    1528468790
867    1528468574
868    1528468524
869    1528467879
870    1528467874
871    1528467826
872    1528466084
873    1528464850
874    1528464646
875    1528464458
876    1528446357
877    1528440705
878    1528440705
Name: timestamp, Length: 879, dtype: int64

In [41]:
dag_hours_details[('timestamp', 'min')]

bid_date    bid_hour
2018-06-08  13          1528440705
            15          1528446357
            20          1528464458
            21          1528467826
            22          1528470014
            23          1528474955
2018-06-09  10          1528513302
            13          1528526354
            15          1528534792
            16          1528535066
            18          1528543105
            19          1528549125
            20          1528549388
            21          1528553770
2018-06-10  3           1528576494
            5           1528585128
            12          1528610126
            13          1528612247
            18          1528630806
            19          1528634535
            20          1528637627
            21          1528639630
            22          1528644832
2018-06-11  3           1528662194
            6           1528674523
                           ...    
2018-07-17  19          1531829233
            20          1531832576

In [37]:
dag_hours_details

timestamp                             price                                        total                                            price_stick      min_ts price
                        count         min         max       min       max      mean           std        sum       min        max      mean       std                            open
bid_date   bid_hour                                                                                                                                                                  
2018-06-08 13               2  1528440705  1528440705  0.000002  0.000002  0.000002  0.000000e+00   0.346220  0.146220   0.200000  0.173110  0.038028  0.000000e+00  1528440705   NaN
           15               1  1528446357  1528446357  0.000002  0.000002  0.000002           NaN   0.287250  0.287250   0.287250  0.287250       NaN  0.000000e+00  1528446357   NaN
           20               4  1528464458  1528466084  0.000002  0.000002  0.000002  5.000000e-09   2.351815  0.309400   1.164800  0.587954  0.403454  1.000000e-08  1528464458   NaN
           21              10  1528467826  1528469540  0.000002  0.000002  0.000002  2.436961e-07   1.653282  0.065792   0.332500  0.165328  0.084238  5.900000e-07  1528467826   NaN
           22               9  1528470014  1528473459  0.000002  0.000002  0.000002  4.795832e-10   1.507221  0.051300   0.592759  0.167469  0.165650  1.000000e-09  1528470014   NaN
           23               1  1528474955  1528474955  0.000002  0.000002  0.000002           NaN   0.151000  0.151000   0.151000  0.151000       NaN  0.000000e+00  1528474955   NaN
2018-06-09 10               8  1528513302  1528515735  0.000002  0.000003  0.000002  9.800753e-08   2.716291  0.150100   0.582805  0.339536  0.151593  2.200000e-07  1528513302   NaN
           13               2  1528526354  1528526892  0.000002  0.000002  0.000002  8.838835e-08   0.710626  0.159250   0.551376  0.355313  0.277275  1.250000e-07  1528526354   NaN
           15               1  1528534792  1528534792  0.000002  0.000002  0.000002           NaN   0.232145  0.232145   0.232145  0.232145       NaN  0.000000e+00  1528534792   NaN
           16               4  1528535066  1528536257  0.000002  0.000002  0.000002  2.982588e-08   0.884572  0.127592   0.281430  0.221143  0.072557  6.500000e-08  1528535066   NaN
           18               1  1528543105  1528543105  0.000002  0.000002  0.000002           NaN   0.320962  0.320962   0.320962  0.320962       NaN  0.000000e+00  1528543105   NaN
           19               2  1528549125  1528549136  0.000002  0.000002  0.000002  1.414214e-09   0.731415  0.274665   0.456750  0.365707  0.128754  2.000000e-09  1528549125   NaN
           20               6  1528549388  1528551854  0.000002  0.000002  0.000002  2.658320e-08   1.458070  0.157000   0.465009  0.243012  0.115122  7.000000e-08  1528549388   NaN
           21               1  1528553770  1528553770  0.000002  0.000002  0.000002           NaN   0.251941  0.251941   0.251941  0.251941       NaN  0.000000e+00  1528553770   NaN
2018-06-10 3                1  1528576494  1528576494  0.000002  0.000002  0.000002           NaN   0.203283  0.203283   0.203283  0.203283       NaN  0.000000e+00  1528576494   NaN
           5                1  1528585128  1528585128  0.000002  0.000002  0.000002           NaN   0.127680  0.127680   0.127680  0.127680       NaN  0.000000e+00  1528585128   NaN
           12               3  1528610126  1528610371  0.000002  0.000002  0.000002  0.000000e+00   1.247750  0.390600   0.461900  0.415917  0.039891  0.000000e+00  1528610126   NaN
           13               2  1528612247  1528612266  0.000002  0.000002  0.000002  1.414214e-08   0.477645  0.200608   0.277037  0.238823  0.054043  2.000000e-08  1528612247   NaN
           18               4  1528630806  1528631727  0.000002  0.000002  0.000002  9.469248e-08   1.254263  0.155000   0.564000  0.313566  0.193161  1.900000e-07  1528630806   NaN
           19   

In [14]:
dag_hist

,amount,buyerFee,date,gasFee,maker,orderHash,price,sellerFee,taker,timestamp,total,transactionHash,type,usdValue,uuid
0,84000,168,2018-07-20 15:43:45,197.32142857142856,0x9dd273165160dc9b6b15189bcb72cfcf88b91160,0x14bcc980b53cb2de61311e90bad9e8dd669a196a574e...,0.0000112,0.0009408,0x91882d719b8850587cd849d79a66aa7d57eb3d06,1532101425,0.9408,0x1fc79b8c91b10d642f76d3eb3cf56870cb3a30ec7e9b...,buy,658.56,af7f81f0-8c33-11e8-b597-3386a0fa2a8f
1,23521.804635669882359493,47.043609271339764719,2018-07-20 15:43:45,197.674418604651167673,0xd850a1ef61ee963b93ef0a4c8b7d0bbc05495b16,0x1e9c6b17b959cbd968878fb223d5a47b587a0782436b...,0.00001118,0.000262973775826789,0x91882d719b8850587cd849d79a66aa7d57eb3d06,1532101425,0.26297377582678927,0x3e315e46ad16d0db48009967f52bb3416e5eecb81aee...,buy,184.081643078752489,af826820-8c33-11e8-b597-3386a0fa2a8f
2,167267.423000000005668864,334.534846000000011338,2018-07-20 15:42:31,198.55513513271776648,0xd84d0a8a1a53a46083931ba3cc99dc09197c199d,0x440488253dc8cc5dabad6483fdd7ec83f137fd54e337...,0.000011130409689595,0.001861754945712745,0x91882d719b8850587cd849d79a66aa7d57eb3d06,1532101351,1.861754945712745216,0x55f6ad4678dcaf5cb7924f3b1956c4af8c0e040f56c4...,buy,1303.2284619989216512,83a7e7c0-8c33-11e8-b597-3386a0fa2a8f
3,14529.914529914529918993,14.529914529914529919,2018-07-20 15:34:23,0.00221,0x91882d719b8850587cd849d79a66aa7d57eb3d06,0xf104d5a08d3ea0fbdbf4701bfc1b2cd6f2ae051faa83...,0.00001053,0.000306,0x93af941a7dcafba72204a37f60a98bfb0bbb5107,1532100863,0.152999999999999999,0x01e46b531965dc9a97cdb41ec328dcba1be33a023487...,sell,107.0999999999999993,60f67c60-8c32-11e8-b597-3386a0fa2a8f
4,5040,5.04,2018-07-20 15:32:11,0.00221,0xd84d0a8a1a53a46083931ba3cc99dc09197c199d,0x5c204b05d0358ef5da5ce7a2f190ea3bf3b252a7eafd...,0.000010539680209863,0.000106239976515419,0xe01a56101c63fd56050445a28b4b6ba346d2a05e,1532100731,0.053119988257709618,0x4b40e56b1351cf921833594883ffa44be11d77aede29...,sell,37.1839917803967326,12643290-8c32-11e8-b597-3386a0fa2a8f
5,36681.482338685194603985,36.681482338685194604,2018-07-20 15:19:39,0.00221,0xe9c5fdc9476df452d031491ad321131bfa7a7e7d,0x7f8e8986ff47a7a4bf882361b3b12d4c72bb7871ffb6...,0.0000116,0.000851010390257497,0x467eb58c894a46e0113089f713f3eef227a5cf7b,1532099979,0.425505195128748257,0xba9d13d1d32ab3139c8645b78f1bfddfff94057c9142...,sell,297.8536365901237799,521225c0-8c30-11e8-b597-3386a0fa2a8f
6,7207.035290074240162079,7.207035290074240162,2018-07-20 15:09:59,0.00221,0xd850a1ef61ee963b93ef0a4c8b7d0bbc05495b16,0x6c42bcec6bc922467829e2083ba955f46d553994264d...,0.00001162,0.000167491500141325,0x04c68b80d63e7b4fbb70901bdfc80a37506c8ded,1532099399,0.08374575007066267,0x2b59d69d46067215121329c4ac577713cc87e8678251...,sell,58.622025049463869,f7ea1ea0-8c2e-11e8-bcc6-4dc86e056f95
7,16338.314667931983102648,16.338314667931983103,2018-07-20 15:06:56,0.00221,0xd850a1ef61ee963b93ef0a4c8b7d0bbc05495b16,0x6c42bcec6bc922467829e2083ba955f46d553994264d...,0.00001162,0.000379702432882739,0x2bdcc5050e7811ac838b1b4ce16ed4133b89fffb,1532099216,0.189851216441369643,0xa3386b294136b44382eb35130d60dfee716babf630a5...,sell,132.8958515089587501,8b63adf0-8c2e-11e8-bcc6-4dc86e056f95
8,151264.117137519788721005,151.264117137519788721,2018-07-20 14:56:02,0.00221,0xe9c5fdc9476df452d031491ad321131bfa7a7e7d,0x7f8e8986ff47a7a4bf882361b3b12d4c72bb7871ffb6...,0.0000116,0.003509327517590459,0xa5cecc3eea916a74db3db813ce404842fa36ba4a,1532098562,1.754663758795229548,0x9e32ba3443334a80198ad58cef69524629dda9bb1101...,sell,1228.2646311566606836,05470ec0-8c2d-11e8-bcc6-4dc86e056f95
9,5250,5.25,2018-07-20 14:53:21,0.002057,0xe9c5fdc9476df452d031491ad321131bfa7a7e7d,0x7f8e8986ff47a7a4bf882361b3b12d4c72bb7871ffb6...,0.0000116,0.0001218,0xa8790283117c04b155cf22941936fa99f6a3de76,1532098401,0.060899999999999999,0x41b5fac9550ebee2425f7ebd677d23d14ab00fdcc78d...,sell,42.6299999999999993,a596e720-8c2c-11e8-bcc6-4dc86e056f95


## Try with ohcl

In [13]:
dag_dt = dag_hist
dag_dt.index = dag_dt['dt']

In [21]:
dag_dt[['date']]

,date
dt,
2018-08-02 17:54:28,2018-08-02 10:54:28
2018-08-02 17:51:11,2018-08-02 10:51:11
2018-08-02 17:49:43,2018-08-02 10:49:43
2018-08-02 17:24:01,2018-08-02 10:24:01
2018-08-02 17:24:01,2018-08-02 10:24:01
2018-08-02 17:24:01,2018-08-02 10:24:01
2018-08-02 17:24:01,2018-08-02 10:24:01
2018-08-02 17:24:01,2018-08-02 10:24:01
2018-08-02 17:23:53,2018-08-02 10:23:53


In [23]:
dag_dt[dag_dt['date'] >= '2018-08-02 10:00:00'][['price']]

,price
dt,
2018-08-02 17:54:28,0.000013
2018-08-02 17:51:11,0.000013
2018-08-02 17:49:43,0.000013
2018-08-02 17:24:01,0.000016
2018-08-02 17:24:01,0.000016
2018-08-02 17:24:01,0.000016
2018-08-02 17:24:01,0.000016
2018-08-02 17:24:01,0.000016
2018-08-02 17:23:53,0.000016


In [33]:
# price_ohlc = dag_dt['price'].resample('15Min').ohlc()

price_ohlc = dag_dt['price'].resample('1H').ohlc()

In [34]:
price_ohlc = price_ohlc.dropna()

In [35]:
price_ohlc

,open,high,low,close
dt,,,,
2018-06-08 13:00:00,0.000002,0.000002,0.000002,0.000002
2018-06-08 15:00:00,0.000002,0.000002,0.000002,0.000002
2018-06-08 20:00:00,0.000002,0.000002,0.000002,0.000002
2018-06-08 21:00:00,0.000002,0.000002,0.000002,0.000002
2018-06-08 22:00:00,0.000002,0.000002,0.000002,0.000002
2018-06-08 23:00:00,0.000002,0.000002,0.000002,0.000002
2018-06-09 10:00:00,0.000002,0.000003,0.000002,0.000003
2018-06-09 13:00:00,0.000002,0.000002,0.000002,0.000002
2018-06-09 15:00:00,0.000002,0.000002,0.000002,0.000002


In [36]:
price_ohlc['body'] = price_ohlc['close'] - price_ohlc['open']
price_ohlc['stick'] = price_ohlc['high'] - price_ohlc['low']

In [37]:
price_ohlc['sb_ratio'] = price_ohlc['stick']/price_ohlc['body']

In [38]:
price_ohlc

,open,high,low,close,body,stick,sb_ratio
dt,,,,,,,
2018-06-08 13:00:00,0.000002,0.000002,0.000002,0.000002,0.000000e+00,0.000000e+00,NaN
2018-06-08 15:00:00,0.000002,0.000002,0.000002,0.000002,0.000000e+00,0.000000e+00,NaN
2018-06-08 20:00:00,0.000002,0.000002,0.000002,0.000002,1.000000e-08,1.000000e-08,1.000000
2018-06-08 21:00:00,0.000002,0.000002,0.000002,0.000002,-5.900000e-07,5.900000e-07,-1.000000
2018-06-08 22:00:00,0.000002,0.000002,0.000002,0.000002,2.000000e-10,1.000000e-09,5.000000
2018-06-08 23:00:00,0.000002,0.000002,0.000002,0.000002,0.000000e+00,0.000000e+00,NaN
2018-06-09 10:00:00,0.000002,0.000003,0.000002,0.000003,2.200000e-07,2.200000e-07,1.000000
2018-06-09 13:00:00,0.000002,0.000002,0.000002,0.000002,-1.250000e-07,1.250000e-07,-1.000000
2018-06-09 15:00:00,0.000002,0.000002,0.000002,0.000002,0.000000e+00,0.000000e+00,NaN


In [39]:
price_ohlc.sort_values(by='sb_ratio', ascending=False)

,open,high,low,close,body,stick,sb_ratio
dt,,,,,,,
2018-07-20 09:00:00,0.000013,0.000014,0.000011,0.000013,0.000000e+00,2.480715e-06,inf
2018-08-02 08:00:00,0.000012,0.000012,0.000012,0.000012,0.000000e+00,4.000000e-08,inf
2018-07-17 15:00:00,0.000003,0.000003,0.000003,0.000003,0.000000e+00,1.999990e-08,inf
2018-07-27 22:00:00,0.000011,0.000011,0.000011,0.000011,0.000000e+00,2.000000e-07,inf
2018-07-18 18:00:00,0.000007,0.000008,0.000006,0.000007,1.000000e-09,1.600000e-06,1600.000000
2018-06-14 19:00:00,0.000002,0.000002,0.000002,0.000002,1.000000e-11,1.000000e-08,1000.000000
2018-07-18 11:00:00,0.000006,0.000008,0.000006,0.000006,1.000000e-08,2.210000e-06,221.000000
2018-07-19 14:00:00,0.000008,0.000008,0.000005,0.000008,2.000000e-08,3.940000e-06,197.000000
2018-07-20 10:00:00,0.000012,0.000013,0.000012,0.000012,2.717488e-08,9.615330e-07,35.383162


In [40]:
price_ohlc.sort_values(by='stick', ascending=False)

,open,high,low,close,body,stick,sb_ratio
dt,,,,,,,
2018-07-19 15:00:00,0.000020,0.000020,0.000005,0.000006,-1.367000e-05,0.000015,-1.084857
2018-07-18 12:00:00,0.000007,0.000014,0.000007,0.000010,3.360000e-06,0.000007,2.217262
2018-08-02 17:00:00,0.000014,0.000017,0.000011,0.000013,-1.100000e-07,0.000005,-48.596295
2018-07-18 13:00:00,0.000009,0.000011,0.000006,0.000007,-2.700000e-06,0.000005,-1.754444
2018-07-26 07:00:00,0.000013,0.000013,0.000009,0.000009,-4.490000e-06,0.000004,-1.000000
2018-07-19 14:00:00,0.000008,0.000008,0.000005,0.000008,2.000000e-08,0.000004,197.000000
2018-07-20 15:00:00,0.000011,0.000013,0.000009,0.000009,-1.994000e-06,0.000004,-1.917703
2018-07-27 12:00:00,0.000008,0.000011,0.000008,0.000010,1.990319e-06,0.000003,1.502432
2018-07-30 14:00:00,0.000013,0.000015,0.000012,0.000012,-1.711726e-06,0.000003,-1.630852


In [44]:
dag_hist[(dag_hist['bid_date'] == '2018-07-19') & (dag_hist['bid_hour'] == 15) & (dag_hist['price'] > 0.0000153)]

,amount,buyerFee,date,gasFee,maker,orderHash,price,sellerFee,taker,timestamp,total,transactionHash,type,usdValue,uuid,dt,bid_hour,bid_minute,bid_date
dt,,,,,,,,,,,,,,,,,,,
2018-07-19 15:11:54,6054545.4546,12109.0909092,2018-07-19 08:11:54,79.770594369129672185,0xae96154e3cd4b738b1881343b12edeb249febd86,0xe3cb6550cdb1f9d883d1d6aa255c58a9d14b4bef9dbc...,0.000019,0.116126181819228,0xfd0b97cfd6862703dec2bab352c5041d00774e8d,1531987914,116.126182,0xa582ca21bc16d4f501ac8c299db171f094ef07d002c2...,buy,81288.3272734596,65c40580-8b2b-11e8-9bd8-89eab97b825e,2018-07-19 15:11:54,15,11,2018-07-19
2018-07-19 15:03:56,4044489.383215369059656218,8088.978766430738119312,2018-07-19 08:03:56,77.350859453993933266,0xae96154e3cd4b738b1881343b12edeb249febd86,0x7af43fb845713fc18413727e249a9e8151f733cfd6c6...,0.000020,0.08,0xfd0b97cfd6862703dec2bab352c5041d00774e8d,1531987436,80.000000,0xc853a12a117cd389a111e10e3f3011473563e475059d...,buy,55999.9999999999999993,49045c70-8b2a-11e8-9bd8-89eab97b825e,2018-07-19 15:03:56,15,3,2018-07-19
2018-07-19 15:03:56,2604290.8432,5208.5816864,2018-07-19 08:03:56,77.664974619287471524,0xae96154e3cd4b738b1881343b12edeb249febd86,0xb61348e2cd136defaef264b0155647723e21a0c6af6c...,0.000020,0.05130452961104,0xfd0b97cfd6862703dec2bab352c5041d00774e8d,1531987436,51.304530,0x83f7d626194d199aef11b5ca9c3f1b1a2050d19d0d1f...,buy,35913.170727728,491413e0-8b2a-11e8-9bd8-89eab97b825e,2018-07-19 15:03:56,15,3,2018-07-19
2018-07-19 15:03:56,550000,1100,2018-07-19 08:03:56,84.9999999999997,0xdb7976ad8fc630f5b5314e702a979053147d0809,0x4db161bcc8fdb71562fbaa5b64d260ec6c83ffa03b40...,0.000018,0.0099,0xfd0b97cfd6862703dec2bab352c5041d00774e8d,1531987436,9.900000,0x191f74bb92f31e2dbf80d87b09c16067e06da4ea97bc...,buy,6930,491db0d0-8b2a-11e8-9bd8-89eab97b825e,2018-07-19 15:03:56,15,3,2018-07-19
2018-07-19 15:03:56,8920,17.84,2018-07-19 08:03:56,89.47368421104954544,0xa5b936cd0f1731748d59c74e9510d8046b1a08ac,0xefba0f3aac0f6906d01c573e712584771e74734d14d9...,0.000017,0.000152531999999108,0xfd0b97cfd6862703dec2bab352c5041d00774e8d,1531987436,0.152532,0xf9b66b3b3c83e2cc9f3dd3fbf69d9f476de5aac19810...,buy,106.7723999993756,49246790-8b2a-11e8-9bd8-89eab97b825e,2018-07-19 15:03:56,15,3,2018-07-19


In [46]:
dag_hist[(dag_hist['bid_date'] == '2018-07-18') & (dag_hist['bid_hour'] == 12) & (dag_hist['price'] > 0.0000116)]

,amount,buyerFee,date,gasFee,maker,orderHash,price,sellerFee,taker,timestamp,total,transactionHash,type,usdValue,uuid,dt,bid_hour,bid_minute,bid_date
dt,,,,,,,,,,,,,,,,,,,
2018-07-18 12:23:28,15211.53846153846147974,30.423076923076922959,2018-07-18 05:23:28,117.692307692307687315,0xb09bf3904d452678d9adb3b42530b12aedab98ed,0x2fd06313376064f344dd276a517a67c4aac7ca36203e...,0.000013,0.00019775,0xd3b6b3dbd8b75674ee41f1e92cd061fe80b499ce,1531891408,0.197750,0xb33d85630e5bf6a55d32882db4904620e181e2b5f26e...,buy,138.4249999999999993,b3a07310-8a4a-11e8-9bd2-ed64a7aa442d,2018-07-18 12:23:28,12,23,2018-07-18
2018-07-18 12:23:02,191444.951843387380832521,382.889903686774761665,2018-07-18 05:23:02,128.67956265769547721,0xc9783407d2f84068075d69a88ad9c13207c2e1ae,0x51593c03cbc5cd816f7cf8df5169b231358d4c9aa5b5...,0.000012,0.002276280477417876,0xba2e39a49735e2d16cd5e2fc929b26deb59c46ec,1531891382,2.276280,0xc109e07a9f78c4f5b26c70811c9a4c486173927f333a...,buy,1593.3963341925131706,a4934cd0-8a4a-11e8-9bd2-ed64a7aa442d,2018-07-18 12:23:02,12,23,2018-07-18
2018-07-18 12:23:02,29931.089583541396189295,59.862179167082792379,2018-07-18 05:23:02,128.712038361234936571,0x3038e4c8fd9b7f379cbdc3dd5c1b4ec85870ddb6,0xe6ecb01d1d51bfc8e34dc62b72f3e74819a6ae9af632...,0.000012,0.000355790861879557,0xba2e39a49735e2d16cd5e2fc929b26deb59c46ec,1531891382,0.355791,0x4df299e22e487e8b9c55b692578d57db3ed5fc7ab333...,buy,249.0536033156896032,a49f0ca0-8a4a-11e8-9bd2-ed64a7aa442d,2018-07-18 12:23:02,12,23,2018-07-18
2018-07-18 12:23:02,100023,200.046,2018-07-18 05:23:02,130.769230769230718742,0x66ea1a4b458d39a0ee6eccc7464450f01669fbcc,0x683d06548d5948b15de3f9c1eb4a3f94f5adb95c5396...,0.000012,0.0011702691,0xba2e39a49735e2d16cd5e2fc929b26deb59c46ec,1531891382,1.170269,0x2f35623846e23c3eab2af073dd9b4b38b4e8cb3e7821...,buy,819.18837,a49ff700-8a4a-11e8-9bd2-ed64a7aa442d,2018-07-18 12:23:02,12,23,2018-07-18
2018-07-18 12:18:24,12565.069148936170141843,25.130138297872340284,2018-07-18 05:18:24,96.453900709219855755,0x66ea1a4b458d39a0ee6eccc7464450f01669fbcc,0x0685873a58d5310c19468cc63e9328ae4df665236e0b...,0.000014,0.000177167475,0xd9afe23909a5377a5295372192f3185827d4def2,1531891104,0.177167,0x4b3073a2584a17a40da7b6784ef8877cd0afe08cb645...,buy,124.0172324999999993,fed185f0-8a49-11e8-9bd2-ed64a7aa442d,2018-07-18 12:18:24,12,18,2018-07-18


## Filter info from hours_details

In [32]:
dag_hours_details.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 376 entries, (2018-06-20, 0) to (2018-07-08, 10)
Data columns (total 9 columns):
(price, min)     376 non-null float64
(price, max)     376 non-null float64
(price, mean)    376 non-null float64
(price, std)     328 non-null float64
(total, sum)     376 non-null float64
(total, min)     376 non-null float64
(total, max)     376 non-null float64
(total, mean)    376 non-null float64
(total, std)     328 non-null float64
dtypes: float64(9)
memory usage: 28.2+ KB


In [100]:
dag_hours_details.loc[('2018-06-28', 11), 'price']

min     0.000062
max     0.000063
mean    0.000063
std     0.000001
Name: (2018-06-28, 11), dtype: float64

In [104]:
dag_hours_details.loc[[('2018-06-28', 11), ('2018-06-20', 11)], 'price']

,,min,max,mean,std
bid_date,bid_hour,,,,
2018-06-28,11,0.000062,0.000063,0.000063,0.000001
2018-06-20,11,0.000056,0.000061,0.000058,0.000002


In [28]:
dag_hours_details['price', 'std'].idxmax()

(u'2018-06-24', 15)

In [34]:
print dag_hours_details.loc['2018-07-04']['price', 'std'].idxmax()

13


In [46]:
dag_hours_details.query('bid_date > "2018-07-04"')

price                                        total                                         
                          min       max      mean           std        sum       min        max      mean       std
bid_date   bid_hour                                                                                                
2018-07-05 0         0.000054  0.000056  0.000055  1.105141e-06   3.445080  0.205031   1.282809  0.689016  0.497733
           1         0.000054  0.000056  0.000055  7.315340e-07   0.504456  0.050103   0.304524  0.126114  0.121099
           2         0.000054  0.000056  0.000055  1.025404e-06   0.951901  0.185960   0.765941  0.475950  0.410108
           4         0.000056  0.000056  0.000056           NaN   0.379226  0.379226   0.379226  0.379226       NaN
           6         0.000055  0.000056  0.000056  1.792907e-07   3.646744  0.190287   1.014625  0.520963  0.335276
           7         0.000056  0.000057  0.000056  5.773503e-07   4.659117  0.200000   4.259117  1.553039  2.343532
           8         0.000058  0.000061  0.000060  1.196735e-06   6.217615  0.061106   1.391145  0.565238  0.411952
           9         0.000061  0.000061  0.000061  7.070399e-17   1.110960  0.122432   0.988528  0.555480  0.612422
           11        0.000056  0.000059  0.000057  1.233311e-06   3.245881  0.289990   1.142067  0.649176  0.422586
           12        0.000055  0.000058  0.000057  1.797498e-06   8.765184  0.679950   3.000000  1.460864  0.877153
           13        0.000054  0.000061  0.000059  2.698127e-06   5.286847  0.151000   1.783298  0.587427  0.494126
           14        0.000059  0.000059  0.000059           NaN   1.165047  1.165047   1.165047  1.165047       NaN
           15        0.000058  0.000061  0.000060  1.765497e-06   0.507129  0.115648   0.269566  0.169043  0.087112
           17        0.000059  0.000059  0.000059  6.243571e-09   0.318083  0.156373   0.161710  0.159041  0.003774
           21        0.000061  0.000061  0.000061  1.729480e-07   2.722983  0.073032   1.206913  0.453831  0.488741
           22        0.000058  0.000058  0.000058           NaN   0.110710  0.110710   0.110710  0.110710       NaN
           23        0.000060  0.000061  0.000061  3.741880e-07   3.169541  0.200000   1.079945  0.528257  0.324806
2018-07-06 0         0.000061  0.000061  0.000061  7.063990e-09   1.665514  0.250000   1.415514  0.832757  0.824143
           1         0.000058  0.000060  0.000059  5.514175e-07  22.711729  0.157120  18.966533  3.785288  7.443385
           2         0.000059  0.000059  0.000059  5.500618e-08   2.010675  0.150180   1.282329  0.502669  0.533334
           3         0.000059  0.000059  0.000059  2.128898e-07   3.946866  0.209808   1.791764  0.986716  0.656855
           5         0.000058  0.000058  0.000058  2.599169e-07   7.275376  0.163590   4.000000  1.212563  1.412560
           7         0.000053  0.000060  0.000054  1.990318e-06   8.062216  0.065906   2.502378  0.732929  0.717798
           8         0.000060  0.000060  0.000060  1.276415e-07   5.133098  0.254647   2.003325  0.733300  0.574013
           9         0.000060  0.000060  0.000060  4.949747e-08   4.017352  1.017352   3.000000  2.008676  1.401944
...                       ...       ...       ...           ...        ...       ...        ...       ...       ...
           17        0.000054  0.000054  0.000054  8.128793e-08   1.169293  0.081267   0.865862  0.389764  0.418288
           20        0.000052  0.000053  0.000053  3.707501e-07   4.461715  0.200378   1.322466  0.637388  0.436455
           21        0.000054  0.000054  0.000054           NaN   0.532014  0.532014   0.532014  0.532014       NaN
           22        0.000051  0.000055  0.000054  1.430345e-06   4.576633  0.171980   1.990127  0.762772  0.696367
           23        0.000052  0.000052  0.000052           NaN   0.162698  0.162698   0.162698  0.162698       NaN
2018-07-07 0         0.000052  0.000056  0.000054  1.668098e-06   2.021471  0.06

In [47]:
print dag_hours_details.query('bid_date > "2018-07-01"')['price', 'std'].idxmax()
print dag_hours_details.query('bid_date > "2018-07-04"')['price', 'std'].idxmax()

(u'2018-07-03', 4)
(u'2018-07-05', 13)


In [29]:
dag_hours_details.loc[[('2018-06-24', 15), ('2018-06-24', 16)], 'price']

min       max      mean       std
bid_date   bid_hour                                        
2018-06-24 15        0.000062  0.000087  0.000076  0.000007
           16        0.000068  0.000080  0.000073  0.000003

In [35]:
dag_hours_details.loc[[('2018-07-04', 9), ('2018-07-04', 13)], ['price','timestamp']]

min       max      mean       std
bid_date   bid_hour                                        
2018-07-04 9         0.000051  0.000055  0.000052  0.000001
           13        0.000050  0.000053  0.000052  0.000002

In [53]:
dag_hours_details.loc[[('2018-07-03', 4), ('2018-07-05', 13)], ['price','timestamp']]

timestamp     price                              
                        count       min       max      mean       std
bid_date   bid_hour                                                  
2018-07-03 4               37  0.000051  0.000063  0.000059  0.000004
2018-07-05 13               9  0.000054  0.000061  0.000059  0.000003

In [40]:
dag_hist[(dag_hist['dt'] >= '2018-07-04 09:00:00') & ((dag_hist['dt'] <= '2018-07-04 14:00:00'))][['price', 'total', 'dt']]

,price,total,dt
293,0.000050,0.149426,2018-07-04 13:46:12
294,0.000053,0.159229,2018-07-04 13:45:37
295,0.000053,0.053266,2018-07-04 13:42:43
296,0.000050,4.802344,2018-07-04 12:31:37
297,0.000051,3.162361,2018-07-04 12:31:03
298,0.000051,1.000000,2018-07-04 12:30:46
299,0.000051,2.034074,2018-07-04 12:30:32
300,0.000052,0.247978,2018-07-04 09:58:51
301,0.000052,2.011993,2018-07-04 09:58:51
302,0.000052,1.005312,2018-07-04 09:58:51


In [48]:
dag_hist[(dag_hist['bid_date'] == '2018-07-03') & (dag_hist['bid_hour'] == 4)][['price', 'total', 'dt']]

,price,total,dt
706,0.000062,0.208184,2018-07-03 04:51:04
707,0.000056,0.188720,2018-07-03 04:49:07
708,0.000063,0.126800,2018-07-03 04:42:22
709,0.000063,0.126800,2018-07-03 04:42:18
710,0.000063,0.317000,2018-07-03 04:42:09
711,0.000063,0.757947,2018-07-03 04:41:32
712,0.000063,0.510053,2018-07-03 04:41:32
713,0.000063,0.570600,2018-07-03 04:41:23
714,0.000063,0.095100,2018-07-03 04:41:12
715,0.000063,1.182875,2018-07-03 04:40:51


In [52]:
len(dag_hist[(dag_hist['bid_date'] == '2018-07-03') & (dag_hist['bid_hour'] == 4)])

37

In [54]:
dag_hist[(dag_hist['bid_date'] == '2018-07-05') & (dag_hist['bid_hour'] == 13)][['price', 'total', 'dt']]

,price,total,dt
175,0.000056,0.231587,2018-07-05 13:42:47
176,0.000054,0.213928,2018-07-05 13:21:53
177,0.000061,0.619671,2018-07-05 13:10:41
178,0.000061,0.151000,2018-07-05 13:10:36
179,0.000061,1.783298,2018-07-05 13:10:29
180,0.000061,0.372071,2018-07-05 13:10:08
181,0.000059,0.725455,2018-07-05 13:09:17
182,0.000059,0.594930,2018-07-05 13:09:09
183,0.000059,0.594908,2018-07-05 13:06:38


## no need for aggregation

In [24]:
dag_hist['bid_date'].value_counts()

2018-06-19    1413
2018-06-20    1040
2018-06-21     541
2018-06-24     465
2018-06-22     260
2018-06-26     245
2018-06-23     218
2018-06-25     201
2018-06-27     127
2018-06-28      31
Name: bid_date, dtype: int64

In [28]:
dag_hist['bid_hour'].value_counts()

8     312
16    290
13    259
12    231
19    223
10    218
9     214
6     213
21    210
20    206
18    204
17    189
3     188
15    187
23    175
5     173
11    150
22    142
4     142
14    140
7     139
2     128
1     120
0      88
Name: bid_hour, dtype: int64

In [39]:
dag_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4541 entries, 0 to 4540
Data columns (total 19 columns):
amount             4541 non-null object
buyerFee           4541 non-null object
date               4541 non-null object
gasFee             4541 non-null object
maker              4541 non-null object
orderHash          4541 non-null object
price              4541 non-null float64
sellerFee          4541 non-null object
taker              4541 non-null object
timestamp          4541 non-null int64
total              4541 non-null float64
transactionHash    4541 non-null object
type               4541 non-null object
usdValue           4541 non-null object
uuid               4541 non-null object
dt                 4541 non-null datetime64[ns]
bid_hour           4541 non-null int64
bid_minute         4541 non-null int64
bid_date           4541 non-null object
dtypes: datetime64[ns](1), float64(2), int64(3), object(13)
memory usage: 674.1+ KB


In [40]:
dag_hist[dag_hist['bid_date'] == "2018-06-27"]

,amount,buyerFee,date,gasFee,maker,orderHash,price,sellerFee,taker,timestamp,total,transactionHash,type,usdValue,uuid,dt,bid_hour,bid_minute,bid_date
31,8735.30043057,17.47060086114,2018-06-27 23:08:25,61.202379105764555529,0x4006352165675fa1b5aaab6f2602320f64798a55,0x961be1c1855d927a0f5b2ec84bf6d69ae762d901783f...,0.000061,0.000533803163989334,0x142e32af9e074867b2240860162fed789239d2ec,1530140905,0.533803,0xcd0bf17e5f13b306b95f65e2633f9d7185e01b15eee9...,buy,373.6622147925334843,fed53420-7a5e-11e8-9a37-e5be5c7eff3b,2018-06-27 23:08:25,23,8,2018-06-27
32,2402.39832438,4.80479664876,2018-06-27 22:55:43,60.229603484086183914,0x4006352165675fa1b5aaab6f2602320f64798a55,0xb124d58a6688cfe74a0b0b921771e8a72baf148e583a...,0.000062,0.000149178629999701,0x72f2c0e11c31ec82e0940353f64122ff68bf91ad,1530140143,0.149179,0x203c54d209cd5a847ac30a5f7c703b5125e09abb85a8...,buy,104.4250409997907539,388cf830-7a5d-11e8-9a37-e5be5c7eff3b,2018-06-27 22:55:43,22,55,2018-06-27
33,49819.10208961,99.63820417922,2018-06-27 22:55:19,60.312852766655572202,0x969c8a30ae88e5e3c2596989367729a465574c56,0xc5364ad1f309580d604e33e6f380f2a4b029a68351ed...,0.000062,0.003089282520526897,0x72f2c0e11c31ec82e0940353f64122ff68bf91ad,1530140119,3.089283,0xabc866f6f57ab24c413dbe1d84b68d355a4a2d2b1e45...,buy,2162.4977643688278979,2a51c7f0-7a5d-11e8-9a37-e5be5c7eff3b,2018-06-27 22:55:19,22,55,2018-06-27
34,32573.28990228,65.14657980456,2018-06-27 22:55:05,60.912052117263828013,0xf4a787267ab7f552868f16a93f098592e6c43e7e,0x14c73d82511af8498ed775d389c6ddb109e404a2ce07...,0.000061,0.001999999999999992,0x72f2c0e11c31ec82e0940353f64122ff68bf91ad,1530140105,2.000000,0x24bc6de3b997c42b22859e7a4d115b5b57fd4fce7a8c...,buy,1399.9999999999944,21bcf920-7a5d-11e8-9a37-e5be5c7eff3b,2018-06-27 22:55:05,22,55,2018-06-27
35,16476.8,32.9536,2018-06-27 22:54:53,61.011419249592166941,0xf2c29d3d23cae388192a562e117b3d0a893e4f58,0x99ba7ad9b835a39b4bd135c403b12eb54a4fbeee896d...,0.000061,0.00101002784,0x72f2c0e11c31ec82e0940353f64122ff68bf91ad,1530140093,1.010028,0x75257c2c5b7f1b26a5727d6ddc9006954ccd9ecf0287...,buy,707.019488,1ae01380-7a5d-11e8-9a37-e5be5c7eff3b,2018-06-27 22:54:53,22,54,2018-06-27
36,34758.06466568,69.51612933136,2018-06-27 22:54:23,61.021373796704160629,0x37ea3845095c933875841aa1eb90558127bd1dc3,0x00c6005dbee2aee54e473c0738169a9acd8f7800d6ca...,0.000061,0.002130321783359527,0x72f2c0e11c31ec82e0940353f64122ff68bf91ad,1530140063,2.130322,0x90b2ddc9565fa31b311f48298be761566c6adca16504...,buy,1491.22524835166904,08f5e500-7a5d-11e8-9a37-e5be5c7eff3b,2018-06-27 22:54:23,22,54,2018-06-27
37,12846.87242399,25.69374484798,2018-06-27 21:54:11,16.641910710676162329,0x9e0a8715fdbae181e92d2149bb00c0da3b381f7f,0x10a1d117dd9cc792f0c6894fecae35f277d74c6eddfe...,0.000061,0.000787398159999945,0x0ddc8cc89f0019c0b67f92dc1be7b02b24665539,1530136451,0.787398,0x3e6b2242a4ebc2ff612f0d8ea6af257c723c431f5ed1...,buy,551.1787119999616267,9fc94750-7a54-11e8-9a37-e5be5c7eff3b,2018-06-27 21:54:11,21,54,2018-06-27
38,3392.99161562,6.78598323124,2018-06-27 21:26:53,16.614908028571023913,0x9e0a8715fdbae181e92d2149bb00c0da3b381f7f,0x339dbd335b816f2d3753ab73278674d8093da9b35b8b...,0.000061,0.000208297959999605,0xe2f8d264acc85351097e86f0de87d558e47b8236,1530134813,0.208298,0x355ca239dd951ab2fce415b481725438a04cbc9e78a3...,buy,145.8085719997233488,cfe58ce0-7a50-11e8-9a37-e5be5c7eff3b,2018-06-27 21:26:53,21,26,2018-06-27
39,14005.58022992,28.01116045984,2018-06-27 21:17:55,16.587708976203010856,0x9e0a8715fdbae181e92d2149bb00c0da3b381f7f,0x50be1176333e81a4cc72504eb564efd5889be58a7c51...,0.000061,0.000861221513773414,0x3179b59e523d1f596039ff2f7804e9db28465e51,1530134275,0.861222,0xdef6b2ae29c1852960ffa6fba4176fc9a4e9326f1ddd...,buy,602.8550596413894759,8f067410-7a4f-11e8-9a37-e5be5c7eff3b,2018-06-27 21:17:55,21,17,2018-06-27
40,12676.48818274,25.35297636548,2018-06-27 21:16:26,16.602262636467050406,0xa0202946f4479ead7565beaa6c4551d5e83dc67d,0xc69311fd232a7a62056610950ba2f0d4c4b483fd1d2f...,0.000061,0.000778810589226186,0

In [46]:
dag_hist[dag_hist['bid_date'] == "2018-06-27"]['total'].sum()

297.0980961530215

In [47]:
dag_hist[dag_hist['bid_date'] == "2018-06-28"]['price'].mean()

6.13612541285343e-05

In [48]:
dag_hist[dag_hist['bid_date'] == "2018-06-27"]['price'].mean()

5.8427166663704974e-05

In [52]:
## get datetime lately
datetime.now()

day_range = pd.date_range(datetime.now(), periods=5, freq='H')

In [68]:
for day in day_range:
    d = day.strftime('%Y-%m-%d')
    h = int(day.strftime('%H'))
    print d, h
    print dag_hist[(dag_hist['bid_date'] == d) & (dag_hist['bid_hour'] == h)]['price'].mean()

2018-06-28 14
nan
2018-06-28 15
nan
2018-06-28 16
nan
2018-06-28 17
nan
2018-06-28 18
nan


In [74]:
dag_hist[(dag_hist['bid_date'] == "2018-06-28") & (dag_hist['bid_hour'] == 10)]['price'].mean()

6.1610709631467e-05